In [1]:
from importlib import reload

import pandas as pd
import numpy as np
from scipy import stats
import statsmodels.api as sm

import matplotlib.pyplot as plt
from matplotlib import rc
import matplotlib.cm as cm

import re

import pickle
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error

C:\Users\339340273\AppData\Local\Continuum\anaconda3\envs\gaku\lib\site-packages\statsmodels\compat\pandas.py:49: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  data_klasses = (pandas.Series, pandas.DataFrame, pandas.Panel)


In [2]:
import probscale
import seaborn as sns

In [3]:
import sys
sys.path.append('C:/dstools')
import sig_test

In [4]:
from pyhive import presto
req_kw = {
  'verify': 'C:/presto/Presto_JDBC_Driver/WDC_CA_bundle.pem'
}

# ETL Script

In [5]:
def get_ghl2_table (table, searchCond, sel, whereAdded = None, fName=None):
    
    con = presto.connect(
      host='bdp-e2e-presto.wdc.com',
      port=8446,
      protocol='https',
      catalog='hive',
      username='gaku.kiuchi@wdc.com',
      requests_kwargs=req_kw
    )

    batchsize = 100

    df = pd.DataFrame({})
    df_sn = pd.read_csv(searchCond[0])
    for strt in range(0, df_sn['sn'].shape[0], batchsize):
        sql = "select %s " % sel
        sql += "from ghl2."+table + " "
        sql += "where product='pdq' "
        sql += "and testcode in (%s) "  % re.sub('[\[\]]', '', '%s' % searchCond[1])
        sql += "and enddt between '%s' and '%s' " % (searchCond[2][0], searchCond[2][1]) 
        sql += "and hddtrial in (%s) "  % re.sub('[\[\]]', '', '%s' % searchCond[3])    
        sql += "and hddsn in (%s) "     % re.sub('[\[\]]', '', '%s' % list(df_sn['sn'].values)[strt:strt+batchsize])
        sql += "and procid in (%s) "    % re.sub('[\[\]]', '', '%s' % searchCond[4])
        sql += "and qualifier in (%s) " % re.sub('[\[\]]', '', '%s' % searchCond[5])
        sql += "and pfcode not in ('9999', '99V6', '99V7') "
        if whereAdded != None:
            sql += "and %s " % whereAdded
        print ('current sql is %s' % sql)
        df = df.append( pd.read_sql(sql, con) )
        print(df.shape)

    con.close()
    
    if fName == None:
        df.to_pickle('df_%s.pkl'%table)
        df.to_csv('df_%s.csv'%table, index=False)
    else:
        df.to_pickle('%s.pkl'%fName)
        df.to_csv('%s.csv'%fName, index=False)
    return(df)

In [6]:
def get_vqaa_table (table, searchCond, sel, whereAdded = None):
    
    con = presto.connect(
      host='bdp-e2e-presto.wdc.com',
      port=8446,
      protocol='https',
      catalog='hive',
      username='gaku.kiuchi@wdc.com',
      requests_kwargs=req_kw
    )

    batchsize = 100

    df = pd.DataFrame({})
    df_sn = pd.read_csv(searchCond[0])
    for strt in range(0, df_sn['sn'].shape[0], batchsize):
        sql = "select %s " % sel
        sql += "from vqaa."+table + " "
        sql += "where product='pdq' "
        sql += "and testpgmver in (%s) "  % re.sub('[\[\]]', '', '%s' % searchCond[1])
        sql += "and enddt between '%s' and '%s' " % (searchCond[2][0], searchCond[2][1]) 
        sql += "and hddtrial in (%s) "  % re.sub('[\[\]]', '', '%s' % searchCond[3])    
        sql += "and hddsn in (%s) "     % re.sub('[\[\]]', '', '%s' % list(df_sn['sn'].values)[strt:strt+batchsize])
        sql += "and procid in (%s) "    % re.sub('[\[\]]', '', '%s' % searchCond[4])
        #sql += "and qualifier in (%s) " % re.sub('[\[\]]', '', '%s' % searchCond[5])
        #sql += "and pfcode not in ('9999', '99V6', '99V7') "
        if whereAdded != None:
            sql += "and %s " % whereAdded
        print ('current sql is %s' % sql)
        df = df.append( pd.read_sql(sql, con) )
        print(df.shape)

    con.close()
        
    df.to_pickle('df_%s.pkl'%table)
    df.to_csv('df_%s.csv'%table, index=False)
    return(df)

In [7]:
def get_sdet_table (table, searchCond, sel, whereAdded = None):
    
    con = presto.connect(
      host='bdp-e2e-presto.wdc.com',
      port=8446,
      protocol='https',
      catalog='hive',
      username='gaku.kiuchi@wdc.com',
      requests_kwargs=req_kw
    )

    batchsize = 100

    df = pd.DataFrame({})
    df_sn = pd.read_csv(searchCond[0])
    for strt in range(0, df_sn['slidersn'].shape[0], batchsize):
        sql = "select %s " % sel
        sql += "from hive."+table + " "
        sql += "where "
        sql += "storeday between '%s' and '%s' " % (searchCond[1][0], searchCond[1][1])
        sql += "and slidersn in (%s) " % re.sub('[\[\]]', '', '%s' % list(df_sn['slidersn'].values)[strt:strt+batchsize])
        if whereAdded != None:
            sql += "and %s " % whereAdded
        print ('current sql is %s' % sql)
        df = df.append( pd.read_sql(sql, con) )
        print(df.shape)

    con.close()
        
    df.to_pickle('df_%s.pkl'%table)
    df.to_csv('df_%s.csv'%table, index=False)
    return(df)

# ETL

In [8]:
#parameters
stpi      = 580.0
dtpi_cmr  = 500.0
dtpi_smr  = 602.0
tgtLbaCmr = 273906721.0
tgtLbaSmr = 333662709.0

In [9]:
#peak current

def IwPeak4G4(iw, ka, kd):
    return((-0.013954886*ka + 1.128161219) * iw + (1.973010793 * ka -1.407847328))

def IwPeak4G4i(iw, ka, kd):
    return((-0.015470889*ka + 1.684161818) * iw + (1.335027916 * ka +0.387637386))


In [10]:
def get_nth_char (strList, index=0):
    a = []
    for q in list(strList):
        a.append(q[index])
    return(a)

In [11]:
snlist_csv   = 'sn_all.csv'
listTestcode = ['PDQC3EKM', 'PDQX073M']
listEnddt    = ['20201204', '20210204']
listHddTrial = ['M1TJ', 'M1U2', 'M1U3', '0000']

# Head Map Check

In [34]:
#Head Map
inputParamGhl2Rmr = [snlist_csv, 
                     listTestcode,
                     listEnddt, 
                     listHddTrial,
                     ['6400'],
                     ['10N0']
                    ]
dfHeadMap = get_ghl2_table('ccb_ci_rmr', inputParamGhl2Rmr, 
                           'hddsn, testcode, mfgid, mfgid_5, hddtrial, lhd, phd, \
                            row_number() over ( partition by hddsn, testcode, lhd order by enddate desc) as row_num')
dfHeadMap.to_pickle('dfHeadMap.pkl')
dfHeadMap = pd.read_pickle('dfHeadMap.pkl').query("row_num==1")
dfHeadMap

current sql is select hddsn, testcode, mfgid, mfgid_5, hddtrial, lhd, phd,                             row_number() over ( partition by hddsn, testcode, lhd order by enddate desc) as row_num from ghl2.ccb_ci_rmr where product='pdq' and testcode in ('PDQC3EKM', 'PDQX073M') and enddt between '20201204' and '20210204' and hddtrial in ('M1TJ', 'M1U2', 'M1U3', '0000') and hddsn in ('2FA0A3AD', '2FA0A43D', '2FA0A4BD', '2FA0A59N', '2FA0A5NN', '2FA0A5VN', '2FA0A2WD', '2FA0A2LD', '2FA0A62N', '2FA09ZSF', '2FA0A63N', '2FA0A6HN', '2FA0A3ZD', '2FA0A37D', '2FA0A2XD', '2FA0A1ZD', '2FA0A4SN', '2FA0A4DD', '2FA0A4HD', '2FA0A3RD', '2FA0A6SN', '2FA0A20D', '2FA0A2RD', '2FA0A17F', '2FA0A5YN', '2FA0A6RN', '2FA0A36D', '2FA0A6GN', '2FA0A3HD', '2FA0A4PN', '2FA0A32D', '2FA0A4MN', '2FA0A6EN', '2FA0A3ND', '2FA0A0VF', '2FA0A3LD', '2FA0A23D', '2FA0A2TD', '2FA0A40D', '2FA0A6XN', '2FA0A3TD', '2FA0A2KD', '2FA0A27D', '2FA0A4ZN', '2FA0A5BN', '2FA0A5XN', '2FA0A35D', '2FA0A18F', '2FA0A03F', '2FA09ZHF', '2FA0A1AF') and proc

,hddsn,testcode,mfgid,mfgid_5,hddtrial,lhd,phd,row_num
0,2FA0A1ZD,PDQX073M,KQBF04,4,M1U2,9,9,1
1,2FA0A5NN,PDQC3EKM,KQBF04,4,0000,6,6,1
2,2FA0A4MN,PDQC3EKM,KQBF04,4,0000,16,16,1
3,2FA0A17F,PDQX073M,KKBF04,4,M1TJ,9,9,1
4,2FA0A03F,PDQX073M,KKBF04,4,M1TJ,6,6,1
...,...,...,...,...,...,...,...,...
1831,2FA0A4BD,PDQC3EKM,KQBF04,4,0000,2,2,1
1832,2FA0A6GN,PDQX073M,KQBF04,4,M1U3,9,9,1
1833,2FA0A0VF,PDQC3EKM,KKBF04,4,0000,17,17,1
1834,2FA0A3RD,PDQC3EKM,KQBF04,4,0000,6,6,1


In [35]:
pd.pivot_table(dfHeadMap.query("lhd==0"), index=['hddsn', 'mfgid_5'], columns=['testcode'], values='hddtrial', aggfunc='count')

,testcode,PDQC3EKM,PDQX073M
hddsn,mfgid_5,,
2FA09ZHF,4,1,1
2FA09ZSF,4,1,1
2FA0A03F,4,1,1
2FA0A0VF,4,1,1
2FA0A17F,4,1,1
2FA0A18F,4,1,1
2FA0A1AF,4,1,1
2FA0A1ZD,4,1,1
2FA0A20D,4,1,1


# SER, OW, Pbo

In [12]:
## SER ##
inputParamGhl2Ser = [snlist_csv, 
                     listTestcode,
                     listEnddt, 
                     listHddTrial,
                     ['6400', '6600', '6800'],
                     ['1000', '1010', 
                      '2000', '2010',
                      '30N0', '3050', '3060', 
                      '40N0', '4050', '4060', 
                      '9070', '9050', '9060', 
                      'K070', 'K050', 'K060',
                      'B0N0', 'B010', 'B020', 
                      'M0N0', 'M010', 'M020',
                     ]
                     #['1000', '1010', '1020', 
                     # '10A0',
                     # '30N0', '3050', '3060']
                    ]
dfSer = get_ghl2_table('ccb_ci_ser', inputParamGhl2Ser, 
                       'hddsn, testcode, qualifier, mfgid, hddtrial, lhd, phd, band, ser, offset, \
                        row_number() over ( partition by hddsn, enddt, testcode, qualifier, lhd, band order by enddate desc) as row_num ',
                       'band < 64'
                      )
dfSer

current sql is select hddsn, testcode, qualifier, mfgid, hddtrial, lhd, phd, band, ser, offset,                         row_number() over ( partition by hddsn, enddt, testcode, qualifier, lhd, band order by enddate desc) as row_num  from ghl2.ccb_ci_ser where product='pdq' and testcode in ('PDQC3EKM', 'PDQX073M') and enddt between '20201204' and '20210204' and hddtrial in ('M1TJ', 'M1U2', 'M1U3', '0000') and hddsn in ('2FA0A3AD', '2FA0A43D', '2FA0A4BD', '2FA0A59N', '2FA0A5NN', '2FA0A5VN', '2FA0A2WD', '2FA0A2LD', '2FA0A62N', '2FA09ZSF', '2FA0A63N', '2FA0A6HN', '2FA0A3ZD', '2FA0A37D', '2FA0A2XD', '2FA0A1ZD', '2FA0A4SN', '2FA0A4DD', '2FA0A4HD', '2FA0A3RD', '2FA0A6SN', '2FA0A20D', '2FA0A2RD', '2FA0A17F', '2FA0A5YN', '2FA0A6RN', '2FA0A36D', '2FA0A6GN', '2FA0A3HD', '2FA0A4PN', '2FA0A32D', '2FA0A4MN', '2FA0A6EN', '2FA0A3ND', '2FA0A0VF', '2FA0A3LD', '2FA0A23D', '2FA0A2TD', '2FA0A40D', '2FA0A6XN', '2FA0A3TD', '2FA0A2KD', '2FA0A27D', '2FA0A4ZN', '2FA0A5BN', '2FA0A5XN', '2FA0A35D', '2FA0A18F', '2

,hddsn,testcode,qualifier,mfgid,hddtrial,lhd,phd,band,ser,offset,row_num
0,2FA09ZHF,PDQX073M,1000,KKBF04,M1TJ,3,3,14,-2.77,-23,1
1,2FA09ZHF,PDQX073M,1000,KKBF04,M1TJ,12,12,24,-2.23,-74,1
2,2FA09ZHF,PDQX073M,3060,KKBF04,M1TJ,16,16,2,-1.85,-54,1
3,2FA09ZHF,PDQX073M,9050,KKBF04,M1TJ,0,0,47,-1.96,-2,1
4,2FA09ZHF,PDQX073M,9050,KKBF04,M1TJ,16,16,39,-1.90,-20,1
...,...,...,...,...,...,...,...,...,...,...,...
923899,2FA0A6XN,PDQX073M,9070,KQBF04,M1U3,14,14,5,-2.02,-193,1
923900,2FA0A6XN,PDQC3EKM,3050,KQBF04,0000,2,2,44,-2.06,-14,1
923901,2FA0A6XN,PDQC3EKM,9060,KQBF04,0000,11,11,56,-1.91,83,1
923902,2FA0A6XN,PDQC3EKM,9060,KQBF04,0000,16,16,10,-1.95,-86,1


In [13]:
## Roller SER ##
inputParamGhl2SerRoller = [snlist_csv, 
                           listTestcode,
                           listEnddt, 
                           listHddTrial,
                           ['6600', '6800'],
                           ['9040', 'B000', 'K040', 'M000']
                          ]
dfSerRoller = get_ghl2_table('ccb_ci_ser', inputParamGhl2SerRoller, 
                       'hddsn, testcode, qualifier, mfgid, hddtrial, lhd, phd, band, offset, \
                        aveperband, maxperband, minperband, rawmodulationstats0_average as aveperband0, rawmodulationstats0_max as maxperband0, rawmodulationstats0_min as minperband0, \
                        row_number() over ( partition by hddsn, enddt, testcode, qualifier, lhd, band order by enddate desc) as row_num ',
                       'band < 64',
                       fName='df_ser_roller'
                      )
dfSerRoller

current sql is select hddsn, testcode, qualifier, mfgid, hddtrial, lhd, phd, band, offset,                         aveperband, maxperband, minperband, rawmodulationstats0_average as aveperband0, rawmodulationstats0_max as maxperband0, rawmodulationstats0_min as minperband0,                         row_number() over ( partition by hddsn, enddt, testcode, qualifier, lhd, band order by enddate desc) as row_num  from ghl2.ccb_ci_ser where product='pdq' and testcode in ('PDQC3EKM', 'PDQX073M') and enddt between '20201204' and '20210204' and hddtrial in ('M1TJ', 'M1U2', 'M1U3', '0000') and hddsn in ('2FA0A3AD', '2FA0A43D', '2FA0A4BD', '2FA0A59N', '2FA0A5NN', '2FA0A5VN', '2FA0A2WD', '2FA0A2LD', '2FA0A62N', '2FA09ZSF', '2FA0A63N', '2FA0A6HN', '2FA0A3ZD', '2FA0A37D', '2FA0A2XD', '2FA0A1ZD', '2FA0A4SN', '2FA0A4DD', '2FA0A4HD', '2FA0A3RD', '2FA0A6SN', '2FA0A20D', '2FA0A2RD', '2FA0A17F', '2FA0A5YN', '2FA0A6RN', '2FA0A36D', '2FA0A6GN', '2FA0A3HD', '2FA0A4PN', '2FA0A32D', '2FA0A4MN', '2FA0A6EN', '2F

,hddsn,testcode,qualifier,mfgid,hddtrial,lhd,phd,band,offset,aveperband,maxperband,minperband,aveperband0,maxperband0,minperband0,row_num
0,2FA0A1AF,PDQC3EKM,9040,KKBF04,0000,6,6,0,-149,-0.950240,-0.854248,-1.111348,-0.95,-0.85,-1.11,1
1,2FA0A2RD,PDQX073M,9040,KQBF04,M1U2,7,7,0,-168,-0.957863,-0.877750,-1.084270,-0.95,-0.87,-1.08,1
2,2FA0A3ND,PDQC3EKM,9040,KQBF04,0000,12,12,0,-60,-0.932797,-0.851184,-1.112293,-0.93,-0.85,-1.11,1
3,2FA0A5YN,PDQC3EKM,9040,KQBF04,0000,17,17,0,-162,-0.924415,-0.801221,-1.149479,-0.92,-0.80,-1.14,1
4,2FA09ZHF,PDQX073M,9040,KKBF04,M1TJ,5,5,0,-99,-0.781417,-0.698229,-0.884893,-0.80,-0.69,-1.17,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1795,2FA0A3LD,PDQC3EKM,9040,KQBF04,0000,6,6,0,-27,-0.909424,-0.818566,-1.057296,-0.90,-0.81,-1.05,1
1796,2FA0A4HD,PDQC3EKM,9040,KQBF04,0000,0,0,0,-83,-0.936321,-0.862253,-1.098170,-0.93,-0.86,-1.09,1
1797,2FA0A5YN,PDQX073M,9040,KQBF04,M1U3,3,3,0,-193,-0.877011,-0.777745,-0.989524,-0.87,-0.77,-0.98,1
1798,2FA0A2RD,PDQC3EKM,9040,KQBF04,0000,0,0,0,-74,-0.894817,-0.826309,-1.049642,-0.89,-0.82,-1.04,1


In [14]:
## OW ##
inputParamGhl2Ow = [snlist_csv, 
                    listTestcode,
                    listEnddt, 
                    listHddTrial,
                    ['6400', '6600', '6800'],
                    ['11N0', '21N0', '31N0', '41N0', '91H0', 'K1H0', 'B0N0', 'M0N0']
                   ]
dfOw = get_ghl2_table('ccb_ci_ow', inputParamGhl2Ow, 
                      'hddsn, enddt, pfcode, mfgid, hddtrial, testcode, qualifier, lhd, phd, band, owvalperp, owvalmaxperp, owvalconv, owvalmaxconv, \
                       row_number() over ( partition by hddsn, enddt, testcode, qualifier, lhd, band order by enddate desc) as row_num',
                      'band < 3'
                     )
dfOw = pd.read_pickle('df_ccb_ci_ow.pkl').query("band < 3 and row_num==1")


current sql is select hddsn, enddt, pfcode, mfgid, hddtrial, testcode, qualifier, lhd, phd, band, owvalperp, owvalmaxperp, owvalconv, owvalmaxconv,                        row_number() over ( partition by hddsn, enddt, testcode, qualifier, lhd, band order by enddate desc) as row_num from ghl2.ccb_ci_ow where product='pdq' and testcode in ('PDQC3EKM', 'PDQX073M') and enddt between '20201204' and '20210204' and hddtrial in ('M1TJ', 'M1U2', 'M1U3', '0000') and hddsn in ('2FA0A3AD', '2FA0A43D', '2FA0A4BD', '2FA0A59N', '2FA0A5NN', '2FA0A5VN', '2FA0A2WD', '2FA0A2LD', '2FA0A62N', '2FA09ZSF', '2FA0A63N', '2FA0A6HN', '2FA0A3ZD', '2FA0A37D', '2FA0A2XD', '2FA0A1ZD', '2FA0A4SN', '2FA0A4DD', '2FA0A4HD', '2FA0A3RD', '2FA0A6SN', '2FA0A20D', '2FA0A2RD', '2FA0A17F', '2FA0A5YN', '2FA0A6RN', '2FA0A36D', '2FA0A6GN', '2FA0A3HD', '2FA0A4PN', '2FA0A32D', '2FA0A4MN', '2FA0A6EN', '2FA0A3ND', '2FA0A0VF', '2FA0A3LD', '2FA0A23D', '2FA0A2TD', '2FA0A40D', '2FA0A6XN', '2FA0A3TD', '2FA0A2KD', '2FA0A27D', '2FA0A4ZN', '

In [15]:
## PBO ##
inputParamGhl2Ow = [snlist_csv, 
                    listTestcode,
                    listEnddt, 
                    listHddTrial,
                    ['6400'],
                    ['3070', '4070']
                   ]
dfPbo = get_ghl2_table('ccb_mi_pbo', inputParamGhl2Ow, 
                      'hddsn, enddt, pfcode, mfgid, hddtrial, testcode, qualifier, lhd, phd, band, mRwSerPerRWNmSlope, mRwSerPerRWDacOffset, \
                       row_number() over ( partition by hddsn, enddt, testcode, qualifier, lhd, band order by enddate desc) as row_num',
                      'band = 0 '
                     )
dfPbo

current sql is select hddsn, enddt, pfcode, mfgid, hddtrial, testcode, qualifier, lhd, phd, band, mRwSerPerRWNmSlope, mRwSerPerRWDacOffset,                        row_number() over ( partition by hddsn, enddt, testcode, qualifier, lhd, band order by enddate desc) as row_num from ghl2.ccb_mi_pbo where product='pdq' and testcode in ('PDQC3EKM', 'PDQX073M') and enddt between '20201204' and '20210204' and hddtrial in ('M1TJ', 'M1U2', 'M1U3', '0000') and hddsn in ('2FA0A3AD', '2FA0A43D', '2FA0A4BD', '2FA0A59N', '2FA0A5NN', '2FA0A5VN', '2FA0A2WD', '2FA0A2LD', '2FA0A62N', '2FA09ZSF', '2FA0A63N', '2FA0A6HN', '2FA0A3ZD', '2FA0A37D', '2FA0A2XD', '2FA0A1ZD', '2FA0A4SN', '2FA0A4DD', '2FA0A4HD', '2FA0A3RD', '2FA0A6SN', '2FA0A20D', '2FA0A2RD', '2FA0A17F', '2FA0A5YN', '2FA0A6RN', '2FA0A36D', '2FA0A6GN', '2FA0A3HD', '2FA0A4PN', '2FA0A32D', '2FA0A4MN', '2FA0A6EN', '2FA0A3ND', '2FA0A0VF', '2FA0A3LD', '2FA0A23D', '2FA0A2TD', '2FA0A40D', '2FA0A6XN', '2FA0A3TD', '2FA0A2KD', '2FA0A27D', '2FA0A4ZN', '2FA0A5B

,hddsn,enddt,pfcode,mfgid,hddtrial,testcode,qualifier,lhd,phd,band,mRwSerPerRWNmSlope,mRwSerPerRWDacOffset,row_num
0,2FA0A3ND,20201205,0000,KQBF04,M1U2,PDQX073M,3070,6,6,0,366.59207,-214.10182,1
1,2FA0A5VN,20201205,0000,KQBF04,M1U3,PDQX073M,3070,1,1,0,309.36035,-221.88797,1
2,2FA0A4SN,20201205,0000,KQBF04,M1U3,PDQX073M,3070,12,12,0,657.57520,-240.76746,1
3,2FA0A5YN,20201205,0000,KQBF04,M1U3,PDQX073M,3070,2,2,0,289.14218,-208.23692,1
4,2FA0A2KD,20210126,0000,KQBF04,0000,PDQC3EKM,3070,7,7,0,440.69830,-228.28150,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1795,2FA0A2RD,20210126,0000,KQBF04,0000,PDQC3EKM,3070,16,16,0,312.68094,-217.61150,1
1796,2FA0A5NN,20201205,0000,KQBF04,M1U3,PDQX073M,3070,8,8,0,457.50897,-222.66156,1
1797,2FA0A4BD,20201205,0000,KQBF04,M1U2,PDQX073M,3070,4,4,0,521.39080,-220.07664,1
1798,2FA0A4MN,20210126,0000,KQBF04,0000,PDQC3EKM,3070,9,9,0,416.87890,-215.71844,1


In [16]:
#TSW2 spec
# 0.0457 * OWC_Worst + 904_ser_roller_max + (0.16+0.194)*u407_rwser_rw_slp_nm_od*1e-3 > -1.5537
#TSWH spec
# 0.0065 * OWC_Worst + k04_ser_roller_max + (0.16+0.194)*u407_rwser_rw_slp_nm_od*1e-3 > -0.66903
coef = [0.0457, 0.0065];      #CMR, SMR
thresh = [-1.5537, -1.6286, -0.66903, -0.68404]; #CMR for SRST, CMR for Final, SMR for SRST, SMR for Final
dfh4S  = 0.16+0.194;     #nm, for SRST
dfh4F  = 0.324;          #nm, for Final

#OW
dfOw = pd.read_pickle('df_ccb_ci_ow.pkl').query("row_num==1 and band==0")
dfOw.loc[:,'firstQual'] = get_nth_char(list(dfOw.qualifier), index=0)
#SER
dfSer = pd.read_pickle('df_ser_roller.pkl').query("row_num==1 and band==0")
dfSer.loc[:,'firstQual'] = get_nth_char(list(dfSer.qualifier), index=0)
#Pbo
dfPbo = pd.read_pickle('df_ccb_mi_pbo.pkl').query("row_num==1 and band==0")

dfTsw = dfOw.query("band==0 and qualifier in ['91H0', 'K1H0', 'B0N0', 'M0N0']").merge(dfSer.query("qualifier in ['9040', 'K040', 'B000', 'M000']"), 
                                                                   left_on =['hddsn', 'mfgid','hddtrial', 'testcode', 'firstQual', 'lhd', 'phd', 'band', 'row_num'],
                                                                   right_on=['hddsn', 'mfgid','hddtrial', 'testcode', 'firstQual', 'lhd', 'phd', 'band', 'row_num'],
                                                                   suffixes=['_ow', '_ser'],
                                                                   how='inner'
                                                                  )
dfTsw=dfTsw.merge(dfPbo,
                  left_on =['hddsn', 'mfgid','hddtrial', 'testcode', 'lhd', 'phd', 'band', 'row_num'],
                  right_on=['hddsn', 'mfgid','hddtrial', 'testcode', 'lhd', 'phd', 'band', 'row_num'],
                  suffixes=['', '_pbo'],
                  how='left'
                 )

#ser with dfh
#SRST
dfTsw.loc[dfTsw.qualifier_ser.str.startswith('9') | dfTsw.qualifier_ser.str.startswith('K'), 'dfh'] = dfh4S
#Final
dfTsw.loc[dfTsw.qualifier_ser.str.startswith('B') | dfTsw.qualifier_ser.str.startswith('M'), 'dfh'] = dfh4F
dfTsw.loc[:,'ser_for_tsw_metric'] = dfTsw.loc[:,'maxperband'].values + dfTsw.loc[:,'mRwSerPerRWNmSlope'].values*dfTsw.loc[:,'dfh'].values*1e-3

#coef
#CMR
dfTsw.loc[dfTsw.qualifier_ser.str.startswith('9') | dfTsw.qualifier_ser.str.startswith('B'),'coef'] = coef[0]
#SMR
dfTsw.loc[dfTsw.qualifier_ser.str.startswith('K') | dfTsw.qualifier_ser.str.startswith('M'),'coef'] = coef[1]

#spec threshold
dfTsw.loc[dfTsw.qualifier_ser.str.startswith('9'), 'thresh'] = thresh[0]
dfTsw.loc[dfTsw.qualifier_ser.str.startswith('B'), 'thresh'] = thresh[1]
dfTsw.loc[dfTsw.qualifier_ser.str.startswith('K'), 'thresh'] = thresh[2]
dfTsw.loc[dfTsw.qualifier_ser.str.startswith('M'), 'thresh'] = thresh[3]

#TSW Metric
dfTsw.loc[:,'tsw_metric'] = dfTsw.loc[:,'owvalmaxconv'].values * dfTsw.loc[:,'coef'].values + dfTsw.loc[:,'ser_for_tsw_metric'].values

dfTsw.to_pickle('dfTsw.pkl')
dfTsw.to_csv('dfTsw.csv')

dfTsw

,hddsn,enddt,pfcode,mfgid,hddtrial,testcode,qualifier_ow,lhd,phd,band,...,enddt_pbo,pfcode_pbo,qualifier,mRwSerPerRWNmSlope,mRwSerPerRWDacOffset,dfh,ser_for_tsw_metric,coef,thresh,tsw_metric
0,2FA0A6SN,20201214,TRW1,KQBF04,M1U3,PDQX073M,91H0,17,17,0,...,20201205,0000,3070,416.84183,-208.24814,0.354,-0.624237,0.0457,-1.5537,-2.088922
1,2FA0A6GN,20201214,TRJ1,KQBF04,M1U3,PDQX073M,91H0,12,12,0,...,20201205,0000,3070,453.58557,-224.32840,0.354,-0.671624,0.0457,-1.5537,-2.105233
2,2FA0A6RN,20210203,0000,KQBF04,0000,PDQC3EKM,91H0,16,16,0,...,20210126,0000,3070,367.52100,-223.03154,0.354,-0.765985,0.0457,-1.5537,-2.259461
3,2FA0A6EN,20210204,0000,KQBF04,0000,PDQC3EKM,91H0,16,16,0,...,20210126,0000,3070,469.47073,-213.30430,0.354,-0.606765,0.0457,-1.5537,-1.933893
4,2FA0A1AF,20201214,TRJ1,KKBF04,M1TJ,PDQX073M,91H0,16,16,0,...,20201205,0000,3070,390.91986,-218.27353,0.354,-0.638826,0.0457,-1.5537,-2.144641
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1795,2FA0A4MN,20201214,TRJ1,KQBF04,M1U3,PDQX073M,91H0,12,12,0,...,20201204,0000,3070,395.78560,-217.46822,0.354,-0.599930,0.0457,-1.5537,-2.127224
1796,2FA0A3ZD,20201214,TRJ1,KQBF04,M1U2,PDQX073M,91H0,14,14,0,...,20201205,0000,3070,329.22147,-217.82294,0.354,-0.650859,0.0457,-1.5537,-2.122399
1797,2FA0A3HD,20210204,0000,KQBF04,0000,PDQC3EKM,91H0,17,17,0,...,20210126,0000,3070,526.19640,-218.40028,0.354,-0.537642,0.0457,-1.5537,-2.028376
1798,2FA0A4BD,20210203,0000,KQBF04,0000,PDQC3EKM,91H0,11,11,0,...,20210126,0000,3070,496.37775,-206.68607,0.354,-0.639552,0.0457,-1.5537,-2.154507


# RSBN

In [17]:
## RSBN ##

inputParamGhl2Rsbn = [snlist_csv, 
                      listTestcode,
                      listEnddt, 
                      listHddTrial,
                      ['6400'],
                      ['1000', '2010']
                     ]
dfRsbn = get_ghl2_table('ccb_ci_crsbn_datareader', inputParamGhl2Rsbn, 
                        'hddsn, enddt, testcode, mfgid, hddtrial, qualifier, phd, lhd, readerindex, datardrs, pridatardr, \
                         row_number() over ( partition by hddsn, testcode, qualifier, lhd, readerindex order by enddate desc) as row_num',
                        'readerindex < 64'
                       )
dfRsbn = pd.read_pickle('df_ccb_ci_crsbn_datareader.pkl')
dfRsbn.loc[:,'datardrs']   = dfRsbn.loc[:,'datardrs'].values.astype(np.int64)
dfRsbn.loc[:,'pridatardr'] = dfRsbn.loc[:,'pridatardr'].values.astype(np.int64)
dfRsbnFlat = pd.pivot_table(dfRsbn.query("readerindex<64 and row_num==1"), 
                            index=['hddsn', 'mfgid', 'testcode', 'lhd', 'phd'], 
                            columns=['qualifier', 'readerindex'], 
                            values=['datardrs', 'pridatardr'], 
                            aggfunc=np.sum).reset_index(drop=False)
cols = [x[0] for x in list(dfRsbnFlat.columns)[:5]]+[str(x[0])+'_'+str(x[1])+'_'+str(x[2]) for x in list(dfRsbnFlat.columns)[5:]]
dfRsbnFlat.columns = cols
dfRsbnFlat.to_pickle('dfRsbnFlat.pkl')
dfRsbnFlat

current sql is select hddsn, enddt, testcode, mfgid, hddtrial, qualifier, phd, lhd, readerindex, datardrs, pridatardr,                          row_number() over ( partition by hddsn, testcode, qualifier, lhd, readerindex order by enddate desc) as row_num from ghl2.ccb_ci_crsbn_datareader where product='pdq' and testcode in ('PDQC3EKM', 'PDQX073M') and enddt between '20201204' and '20210204' and hddtrial in ('M1TJ', 'M1U2', 'M1U3', '0000') and hddsn in ('2FA0A3AD', '2FA0A43D', '2FA0A4BD', '2FA0A59N', '2FA0A5NN', '2FA0A5VN', '2FA0A2WD', '2FA0A2LD', '2FA0A62N', '2FA09ZSF', '2FA0A63N', '2FA0A6HN', '2FA0A3ZD', '2FA0A37D', '2FA0A2XD', '2FA0A1ZD', '2FA0A4SN', '2FA0A4DD', '2FA0A4HD', '2FA0A3RD', '2FA0A6SN', '2FA0A20D', '2FA0A2RD', '2FA0A17F', '2FA0A5YN', '2FA0A6RN', '2FA0A36D', '2FA0A6GN', '2FA0A3HD', '2FA0A4PN', '2FA0A32D', '2FA0A4MN', '2FA0A6EN', '2FA0A3ND', '2FA0A0VF', '2FA0A3LD', '2FA0A23D', '2FA0A2TD', '2FA0A40D', '2FA0A6XN', '2FA0A3TD', '2FA0A2KD', '2FA0A27D', '2FA0A4ZN', '2FA0A5BN', '2

,hddsn,mfgid,testcode,lhd,phd,datardrs_1000_0,datardrs_1000_1,datardrs_1000_2,datardrs_1000_3,datardrs_1000_4,...,pridatardr_1000_54,pridatardr_1000_55,pridatardr_1000_56,pridatardr_1000_57,pridatardr_1000_58,pridatardr_1000_59,pridatardr_1000_60,pridatardr_1000_61,pridatardr_1000_62,pridatardr_1000_63
0,2FA09ZHF,KKBF04,PDQC3EKM,0,0,3,3,3,3,3,...,0,0,0,0,0,0,0,0,0,0
1,2FA09ZHF,KKBF04,PDQC3EKM,1,1,3,3,3,3,3,...,0,0,0,0,0,0,0,0,0,0
2,2FA09ZHF,KKBF04,PDQC3EKM,2,2,3,3,3,3,3,...,1,1,1,1,1,1,1,1,1,1
3,2FA09ZHF,KKBF04,PDQC3EKM,3,3,3,3,3,3,3,...,0,0,0,0,0,0,0,0,0,0
4,2FA09ZHF,KKBF04,PDQC3EKM,4,4,3,3,3,3,3,...,1,1,1,1,1,1,1,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1813,2FA0A6XN,KQBF04,PDQX073M,13,13,3,3,3,3,3,...,0,0,0,0,0,0,0,0,0,0
1814,2FA0A6XN,KQBF04,PDQX073M,14,14,3,3,3,3,3,...,0,0,0,0,0,0,0,0,0,0
1815,2FA0A6XN,KQBF04,PDQX073M,15,15,3,3,3,3,3,...,1,1,1,1,1,1,1,1,1,1
1816,2FA0A6XN,KQBF04,PDQX073M,16,16,3,3,3,3,3,...,1,1,1,1,1,1,1,1,1,1


In [18]:
## MCW ##
inputParamGhl2Mcw = [snlist_csv, 
                      listTestcode,
                      listEnddt, 
                      listHddTrial,
                      ['6400'],
                      ['10N0', '2300']
                     ]
dfMcw = get_ghl2_table('ccb_ci_mcw', inputParamGhl2Mcw, 
                        'hddsn, enddt, testcode, mfgid, hddtrial, qualifier, band, phd, lhd, mcw, mcw_smr, \
                         row_number() over ( partition by hddsn, testcode, qualifier, lhd, band order by enddate desc) as row_num')
dfMcw = pd.read_pickle('df_ccb_ci_mcw.pkl').query("row_num==1")
dfMcw.loc[:,'mcw_nm']     = dfMcw.loc[:,'mcw'].values     * (1/100*25.4/dtpi_cmr*1e3)
dfMcw.loc[:,'mcw_smr_nm'] = dfMcw.loc[:,'mcw_smr'].values * (1/4096*25.4/dtpi_smr*1e3)
dfMcw.to_pickle('dfMcw.pkl')
dfMcw

current sql is select hddsn, enddt, testcode, mfgid, hddtrial, qualifier, band, phd, lhd, mcw, mcw_smr,                          row_number() over ( partition by hddsn, testcode, qualifier, lhd, band order by enddate desc) as row_num from ghl2.ccb_ci_mcw where product='pdq' and testcode in ('PDQC3EKM', 'PDQX073M') and enddt between '20201204' and '20210204' and hddtrial in ('M1TJ', 'M1U2', 'M1U3', '0000') and hddsn in ('2FA0A3AD', '2FA0A43D', '2FA0A4BD', '2FA0A59N', '2FA0A5NN', '2FA0A5VN', '2FA0A2WD', '2FA0A2LD', '2FA0A62N', '2FA09ZSF', '2FA0A63N', '2FA0A6HN', '2FA0A3ZD', '2FA0A37D', '2FA0A2XD', '2FA0A1ZD', '2FA0A4SN', '2FA0A4DD', '2FA0A4HD', '2FA0A3RD', '2FA0A6SN', '2FA0A20D', '2FA0A2RD', '2FA0A17F', '2FA0A5YN', '2FA0A6RN', '2FA0A36D', '2FA0A6GN', '2FA0A3HD', '2FA0A4PN', '2FA0A32D', '2FA0A4MN', '2FA0A6EN', '2FA0A3ND', '2FA0A0VF', '2FA0A3LD', '2FA0A23D', '2FA0A2TD', '2FA0A40D', '2FA0A6XN', '2FA0A3TD', '2FA0A2KD', '2FA0A27D', '2FA0A4ZN', '2FA0A5BN', '2FA0A5XN', '2FA0A35D', '2FA0A18F', '

,hddsn,enddt,testcode,mfgid,hddtrial,qualifier,band,phd,lhd,mcw,mcw_smr,row_num,mcw_nm,mcw_smr_nm
0,2FA0A1ZD,20210126,PDQC3EKM,KQBF04,0000,10N0,1,8,8,112.011719,0,1,56.901953,0.0
1,2FA0A3AD,20201204,PDQX073M,KQBF04,M1U2,10N0,1,9,9,116.381836,0,1,59.121973,0.0
2,2FA0A2WD,20210126,PDQC3EKM,KQBF04,0000,10N0,1,2,2,108.032227,0,1,54.880371,0.0
3,2FA0A6XN,20201205,PDQX073M,KQBF04,M1U3,10N0,2,17,17,117.895508,0,1,59.890918,0.0
4,2FA0A4DD,20210126,PDQC3EKM,KQBF04,0000,10N0,2,11,11,115.454102,0,1,58.650684,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5449,2FA0A4DD,20210126,PDQC3EKM,KQBF04,0000,10N0,1,0,0,110.473633,0,1,56.120605,0.0
5450,2FA0A5BN,20210126,PDQC3EKM,KQBF04,0000,10N0,0,11,11,119.091797,0,1,60.498633,0.0
5451,2FA0A6RN,20210126,PDQC3EKM,KQBF04,0000,10N0,0,10,10,127.539062,0,1,64.789844,0.0
5452,2FA0A2RD,20210126,PDQC3EKM,KQBF04,0000,10N0,0,4,4,124.121094,0,1,63.053516,0.0


In [19]:
## MRW/MWW ##
inputParamGhl2Mrw = [snlist_csv, 
                      listTestcode,
                      listEnddt, 
                      listHddTrial,
                      ['6400'],
                      ['1000', '1010', '2000', '2010']
                     ]
dfMrw = get_ghl2_table('ccb_ci_mrw', inputParamGhl2Mrw, 
                       'hddsn, enddt, testcode, mfgid, hddtrial, qualifier, band, phd, lhd, mrw, mww, \
                        row_number() over ( partition by hddsn, testcode, qualifier, lhd, band order by enddate desc) as row_num',
                       'band < 3'
                      )
dfMrw = pd.read_pickle('df_ccb_ci_mrw.pkl').query("row_num==1")
dfMrw.loc[dfMrw.qualifier.str.startswith('1'), 'scale'] = 1/100*25.4/dtpi_cmr*1e3
dfMrw.loc[dfMrw.qualifier.str.startswith('2'), 'scale'] = 1/100*25.4/dtpi_smr*1e3
dfMrw.loc[:,'mrw_nm'] = dfMrw.loc[:,'mrw'].values * dfMrw.loc[:, 'scale'].values
dfMrw.loc[:,'mww_nm'] = dfMrw.loc[:,'mww'].values * dfMrw.loc[:, 'scale'].values
dfMrw.to_pickle('dfMrw.pkl')
dfMrw

current sql is select hddsn, enddt, testcode, mfgid, hddtrial, qualifier, band, phd, lhd, mrw, mww,                         row_number() over ( partition by hddsn, testcode, qualifier, lhd, band order by enddate desc) as row_num from ghl2.ccb_ci_mrw where product='pdq' and testcode in ('PDQC3EKM', 'PDQX073M') and enddt between '20201204' and '20210204' and hddtrial in ('M1TJ', 'M1U2', 'M1U3', '0000') and hddsn in ('2FA0A3AD', '2FA0A43D', '2FA0A4BD', '2FA0A59N', '2FA0A5NN', '2FA0A5VN', '2FA0A2WD', '2FA0A2LD', '2FA0A62N', '2FA09ZSF', '2FA0A63N', '2FA0A6HN', '2FA0A3ZD', '2FA0A37D', '2FA0A2XD', '2FA0A1ZD', '2FA0A4SN', '2FA0A4DD', '2FA0A4HD', '2FA0A3RD', '2FA0A6SN', '2FA0A20D', '2FA0A2RD', '2FA0A17F', '2FA0A5YN', '2FA0A6RN', '2FA0A36D', '2FA0A6GN', '2FA0A3HD', '2FA0A4PN', '2FA0A32D', '2FA0A4MN', '2FA0A6EN', '2FA0A3ND', '2FA0A0VF', '2FA0A3LD', '2FA0A23D', '2FA0A2TD', '2FA0A40D', '2FA0A6XN', '2FA0A3TD', '2FA0A2KD', '2FA0A27D', '2FA0A4ZN', '2FA0A5BN', '2FA0A5XN', '2FA0A35D', '2FA0A18F', '2FA0A

,hddsn,enddt,testcode,mfgid,hddtrial,qualifier,band,phd,lhd,mrw,mww,row_num,scale,mrw_nm,mww_nm
0,2FA0A1ZD,20210126,PDQC3EKM,KQBF04,0000,1000,2,15,15,49.295010,83.187950,1,0.508,25.041865,42.259479
1,2FA0A2KD,20210126,PDQC3EKM,KQBF04,0000,1000,2,11,11,52.172695,94.131714,1,0.508,26.503729,47.818911
2,2FA0A2RD,20210126,PDQC3EKM,KQBF04,0000,1010,0,12,12,48.630962,89.504530,1,0.508,24.704529,45.468301
3,2FA0A2TD,20201205,PDQX073M,KQBF04,M1U2,1010,2,14,14,46.941723,94.258840,1,0.508,23.846395,47.883491
4,2FA0A37D,20201205,PDQX073M,KQBF04,M1U2,1010,1,7,7,46.025547,94.597740,1,0.508,23.380978,48.055652
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10903,2FA0A3TD,20201204,PDQX073M,KQBF04,M1U2,1010,0,15,15,49.014565,96.244380,1,0.508,24.899399,48.892145
10904,2FA0A43D,20210126,PDQC3EKM,KQBF04,0000,1000,1,11,11,49.949394,92.602990,1,0.508,25.374292,47.042319
10905,2FA0A59N,20201205,PDQX073M,KQBF04,M1U3,1000,0,3,3,55.860622,106.141870,1,0.508,28.377196,53.920070
10906,2FA0A5NN,20210126,PDQC3EKM,KQBF04,0000,1000,2,7,7,53.247220,88.146675,1,0.508,27.049588,44.778511


In [20]:
## AMP ##
inputParamGhl2Amp = [snlist_csv, 
                      listTestcode,
                      listEnddt, 
                      listHddTrial,
                      ['6400', '6600', '6800'],
                      ['1000', '1010', '2000', '2010', 
                       '3000', '3010', '4000', '4010', 
                       '9000', '9010', 'K000', 'K010',
                       'B000', 'B010', 'M000', 'M010' 
                      ]
                     ]
dfAmp = get_ghl2_table('ccb_ci_amp', 
                       inputParamGhl2Amp, 
                       #'hddsn, pfcode, testcode, mfgid, hddtrial, qualifier, band, phd, lhd, MRResistance, InputImpedance, SelectReaderGain, CalcedAeGain, AeGainCompensation_ResolutionAVG, AeGainCompensation_Amplitude1AVG, AeGainCompensation_Amplitude2AVG ')
                       'hddsn, enddt, testcode, mfgid, hddtrial, qualifier, band, phd, lhd, AeGainCompensation_ResolutionAVG as res, AeGainCompensation_Amplitude1AVG as amp2t, AeGainCompensation_Amplitude2AVG as amp10t, \
                        row_number() over ( partition by hddsn, testcode, qualifier, lhd, band order by enddate desc) as row_num',
                       'band < 3'
                      )
dfAmp = pd.read_pickle('df_ccb_ci_amp.pkl')
dfAmpFlat = pd.pivot_table(dfAmp.query("band<3 and row_num==1"), 
                           index=['hddsn', 'mfgid', 'testcode', 'lhd', 'phd'], 
                           columns=['qualifier', 'band'], 
                           values=['res', 'amp2t', 'amp10t'], 
                           aggfunc=np.sum).reset_index(drop=False)
cols = [x[0] for x in list(dfAmpFlat.columns)[:5]]+[str(x[0])+'_'+str(x[1])+'_'+str(x[2]) for x in list(dfAmpFlat.columns)[5:]]
dfAmpFlat.columns = cols
dfAmpFlat

current sql is select hddsn, enddt, testcode, mfgid, hddtrial, qualifier, band, phd, lhd, AeGainCompensation_ResolutionAVG as res, AeGainCompensation_Amplitude1AVG as amp2t, AeGainCompensation_Amplitude2AVG as amp10t,                         row_number() over ( partition by hddsn, testcode, qualifier, lhd, band order by enddate desc) as row_num from ghl2.ccb_ci_amp where product='pdq' and testcode in ('PDQC3EKM', 'PDQX073M') and enddt between '20201204' and '20210204' and hddtrial in ('M1TJ', 'M1U2', 'M1U3', '0000') and hddsn in ('2FA0A3AD', '2FA0A43D', '2FA0A4BD', '2FA0A59N', '2FA0A5NN', '2FA0A5VN', '2FA0A2WD', '2FA0A2LD', '2FA0A62N', '2FA09ZSF', '2FA0A63N', '2FA0A6HN', '2FA0A3ZD', '2FA0A37D', '2FA0A2XD', '2FA0A1ZD', '2FA0A4SN', '2FA0A4DD', '2FA0A4HD', '2FA0A3RD', '2FA0A6SN', '2FA0A20D', '2FA0A2RD', '2FA0A17F', '2FA0A5YN', '2FA0A6RN', '2FA0A36D', '2FA0A6GN', '2FA0A3HD', '2FA0A4PN', '2FA0A32D', '2FA0A4MN', '2FA0A6EN', '2FA0A3ND', '2FA0A0VF', '2FA0A3LD', '2FA0A23D', '2FA0A2TD', '2FA0A40

,hddsn,mfgid,testcode,lhd,phd,amp10t_1000_0,amp10t_1000_1,amp10t_1000_2,amp10t_1010_0,amp10t_1010_1,...,res_3000_2,res_3010_0,res_3010_1,res_3010_2,res_9000_0,res_9000_1,res_9000_2,res_9010_0,res_9010_1,res_9010_2
0,2FA09ZHF,KKBF04,PDQC3EKM,0,0,23.486115,24.559376,24.323599,19.343260,20.075957,...,31.392761,35.956743,31.116971,33.279623,36.803740,30.339153,31.095291,36.772150,31.122710,33.096346
1,2FA09ZHF,KKBF04,PDQC3EKM,1,1,33.249799,35.368817,34.862487,17.935581,19.239106,...,38.867386,51.646510,41.157147,37.917002,47.835285,38.627850,38.900234,50.331799,40.582557,40.261369
2,2FA09ZHF,KKBF04,PDQC3EKM,2,2,25.408424,26.895623,26.951526,27.782562,28.208622,...,32.000866,38.431746,34.471460,32.521847,36.204373,30.930459,30.711915,36.491804,34.050481,31.335696
3,2FA09ZHF,KKBF04,PDQC3EKM,3,3,40.203711,42.257290,42.796759,28.361315,30.013650,...,38.408315,42.459431,35.096650,34.560591,46.683890,36.767274,38.056305,43.242302,34.474082,33.397505
4,2FA09ZHF,KKBF04,PDQC3EKM,4,4,35.949985,36.556051,35.958891,24.074428,25.489973,...,36.511633,38.634661,33.583851,34.407186,42.044777,33.902795,36.989326,39.111347,32.811847,34.252876
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1813,2FA0A6XN,KQBF04,PDQX073M,13,13,31.855727,33.301288,31.860882,13.286598,13.939316,...,38.035478,45.656277,37.076286,35.001989,45.670975,38.573534,37.954714,44.833137,37.529143,35.032955
1814,2FA0A6XN,KQBF04,PDQX073M,14,14,28.879541,29.547492,27.610591,21.038508,21.299549,...,30.895074,39.001488,33.892960,33.796276,34.654146,29.208273,31.270210,37.150041,32.485376,33.442296
1815,2FA0A6XN,KQBF04,PDQX073M,15,15,25.987370,26.753019,26.078095,19.449048,19.893638,...,27.281155,26.679977,32.374441,31.779970,22.291947,25.775176,25.536622,27.559865,32.205781,32.390796
1816,2FA0A6XN,KQBF04,PDQX073M,16,16,30.703841,31.274286,29.561688,27.674906,28.102020,...,30.350970,36.913481,30.499176,31.519711,34.402311,28.369407,31.259446,34.969915,29.578114,30.998155


In [21]:
## Asym ##

inputParamGhl2Asym = [snlist_csv, 
                      listTestcode,
                      listEnddt, 
                      listHddTrial,
                      ['6400', '6600', '6800'],
                      ['1000', '1010', '2000', '2010',
                       '3000', '3010', '4000', '4010', 
                       '9000', '9010', 'K000', 'K010',
                       'B000', 'B010', 'M000', 'M010'
                      ]
                     ]
dfAsym = get_ghl2_table('ccb_ci_asm', 
                       inputParamGhl2Asym, 
                       'hddsn, enddt, testcode, mfgid, hddtrial, qualifier, band, phd, lhd, asymmetry, \
                        row_number() over ( partition by hddsn, testcode, qualifier, lhd, band order by enddate desc) as row_num',
                       'band < 3'
                       )
"""
dfAsym = pd.read_pickle('df_ccb_ci_asm.pkl')
dfAsymFlat = pd.pivot_table(dfAsym.query("band<3 and row_num==1"), 
                           index=['hddsn', 'mfgid', 'testcode', 'lhd', 'phd'], 
                           columns=['qualifier', 'band'], 
                           values=['asymmetry'], 
                           aggfunc=np.sum).reset_index(drop=False)
cols = [x[0] for x in list(dfAsymFlat.columns)[:5]]+[str(x[0])+'_'+str(x[1])+'_'+str(x[2]) for x in list(dfAsymFlat.columns)[5:]]
dfAsymFlat.columns = cols
dfAsymFlat
"""

current sql is select hddsn, enddt, testcode, mfgid, hddtrial, qualifier, band, phd, lhd, asymmetry,                         row_number() over ( partition by hddsn, testcode, qualifier, lhd, band order by enddate desc) as row_num from ghl2.ccb_ci_asm where product='pdq' and testcode in ('PDQC3EKM', 'PDQX073M') and enddt between '20201204' and '20210204' and hddtrial in ('M1TJ', 'M1U2', 'M1U3', '0000') and hddsn in ('2FA0A3AD', '2FA0A43D', '2FA0A4BD', '2FA0A59N', '2FA0A5NN', '2FA0A5VN', '2FA0A2WD', '2FA0A2LD', '2FA0A62N', '2FA09ZSF', '2FA0A63N', '2FA0A6HN', '2FA0A3ZD', '2FA0A37D', '2FA0A2XD', '2FA0A1ZD', '2FA0A4SN', '2FA0A4DD', '2FA0A4HD', '2FA0A3RD', '2FA0A6SN', '2FA0A20D', '2FA0A2RD', '2FA0A17F', '2FA0A5YN', '2FA0A6RN', '2FA0A36D', '2FA0A6GN', '2FA0A3HD', '2FA0A4PN', '2FA0A32D', '2FA0A4MN', '2FA0A6EN', '2FA0A3ND', '2FA0A0VF', '2FA0A3LD', '2FA0A23D', '2FA0A2TD', '2FA0A40D', '2FA0A6XN', '2FA0A3TD', '2FA0A2KD', '2FA0A27D', '2FA0A4ZN', '2FA0A5BN', '2FA0A5XN', '2FA0A35D', '2FA0A18F', '2FA0

'\ndfAsym = pd.read_pickle(\'df_ccb_ci_asm.pkl\')\ndfAsymFlat = pd.pivot_table(dfAsym.query("band<3 and row_num==1"), \n                           index=[\'hddsn\', \'mfgid\', \'testcode\', \'lhd\', \'phd\'], \n                           columns=[\'qualifier\', \'band\'], \n                           values=[\'asymmetry\'], \n                           aggfunc=np.sum).reset_index(drop=False)\ncols = [x[0] for x in list(dfAsymFlat.columns)[:5]]+[str(x[0])+\'_\'+str(x[1])+\'_\'+str(x[2]) for x in list(dfAsymFlat.columns)[5:]]\ndfAsymFlat.columns = cols\ndfAsymFlat\n'

In [22]:
## RWIP ##

inputParamGhl2Rwip = [snlist_csv, 
                      listTestcode,
                      listEnddt, 
                      listHddTrial,
                      ['6400', '6600', '6800'],
                      ['1000', '1010', '2000', '2010', 
                       '3000', '3010', '4000', '4010', 
                       '9000', '9010', 'K000', 'K010',
                       'B000', 'B010', 'M000', 'M010'
                      ]
                     ]
dfRwip = get_ghl2_table('ccb_ci_rwip', 
                        inputParamGhl2Rwip, 
                       'hddsn, enddt, testcode, mfgid, hddtrial, qualifier, band, phd, lhd, snrtotal, snrsystem, snrmedia, \
                        row_number() over ( partition by hddsn, testcode, qualifier, lhd, band order by enddate desc) as row_num',
                        'band < 3'
                       )
"""
dfRwip = pd.read_pickle('df_ccb_ci_rwip.pkl').astype({'snrtotal': 'float64', 'snrsystem': 'float64', 'snrmedia': 'float64'})
dfRwipFlat = pd.pivot_table(dfRwip.query("band<3 and row_num==1"), 
                           index=['hddsn', 'mfgid', 'testcode', 'lhd', 'phd'], 
                           columns=['qualifier', 'band'], 
                           values=['snrtotal', 'snrsystem', 'snrmedia'], 
                           aggfunc=np.sum).reset_index(drop=False)
cols = [x[0] for x in list(dfRwipFlat.columns)[:5]]+[str(x[0])+'_'+str(x[1])+'_'+str(x[2]) for x in list(dfRwipFlat.columns)[5:]]
dfRwipFlat.columns = cols
dfRwipFlat
"""

current sql is select hddsn, enddt, testcode, mfgid, hddtrial, qualifier, band, phd, lhd, snrtotal, snrsystem, snrmedia,                         row_number() over ( partition by hddsn, testcode, qualifier, lhd, band order by enddate desc) as row_num from ghl2.ccb_ci_rwip where product='pdq' and testcode in ('PDQC3EKM', 'PDQX073M') and enddt between '20201204' and '20210204' and hddtrial in ('M1TJ', 'M1U2', 'M1U3', '0000') and hddsn in ('2FA0A3AD', '2FA0A43D', '2FA0A4BD', '2FA0A59N', '2FA0A5NN', '2FA0A5VN', '2FA0A2WD', '2FA0A2LD', '2FA0A62N', '2FA09ZSF', '2FA0A63N', '2FA0A6HN', '2FA0A3ZD', '2FA0A37D', '2FA0A2XD', '2FA0A1ZD', '2FA0A4SN', '2FA0A4DD', '2FA0A4HD', '2FA0A3RD', '2FA0A6SN', '2FA0A20D', '2FA0A2RD', '2FA0A17F', '2FA0A5YN', '2FA0A6RN', '2FA0A36D', '2FA0A6GN', '2FA0A3HD', '2FA0A4PN', '2FA0A32D', '2FA0A4MN', '2FA0A6EN', '2FA0A3ND', '2FA0A0VF', '2FA0A3LD', '2FA0A23D', '2FA0A2TD', '2FA0A40D', '2FA0A6XN', '2FA0A3TD', '2FA0A2KD', '2FA0A27D', '2FA0A4ZN', '2FA0A5BN', '2FA0A5XN', '2FA0A35

'\ndfRwip = pd.read_pickle(\'df_ccb_ci_rwip.pkl\').astype({\'snrtotal\': \'float64\', \'snrsystem\': \'float64\', \'snrmedia\': \'float64\'})\ndfRwipFlat = pd.pivot_table(dfRwip.query("band<3 and row_num==1"), \n                           index=[\'hddsn\', \'mfgid\', \'testcode\', \'lhd\', \'phd\'], \n                           columns=[\'qualifier\', \'band\'], \n                           values=[\'snrtotal\', \'snrsystem\', \'snrmedia\'], \n                           aggfunc=np.sum).reset_index(drop=False)\ncols = [x[0] for x in list(dfRwipFlat.columns)[:5]]+[str(x[0])+\'_\'+str(x[1])+\'_\'+str(x[2]) for x in list(dfRwipFlat.columns)[5:]]\ndfRwipFlat.columns = cols\ndfRwipFlat\n'

# AIw

In [26]:
#AIw
inputParamGhl2AIw = [snlist_csv, 
                      listTestcode,
                      listEnddt, 
                      listHddTrial,
                      ['6400'],
                      ['10N0', '11N0', '30N0', '20N0', '40N0']
                     ]
dfAIw = get_ghl2_table('ccb_ci_aiw_summary', 
                        inputParamGhl2AIw, 
                       'hddsn, enddate, pfcode, testcode, mfgid, mfgid_5, hddtrial, qualifier, lhd, phd, readerselection, \
                        cmdexecutiontimeinmilliseconds as time, band, cylinder, meastemp, \
                        writecurrent as iw, kickamplitude as ka, kickduration as kd, bestser, readoffset, enddt, \
                        row_number() over ( partition by hddsn, testcode, qualifier, lhd, band order by enddate desc) as row_num'
                       )
dfAIw.loc[:,'mfgid_5'] = dfAIw['mfgid_5'].values.astype('int')
#Gen4
dfAIw.loc[dfAIw['mfgid_5'] < 4,'coef_a'] = -0.013954886
dfAIw.loc[dfAIw['mfgid_5'] < 4,'coef_b'] = 1.128161219
dfAIw.loc[dfAIw['mfgid_5'] < 4,'coef_c'] = 1.973010793
dfAIw.loc[dfAIw['mfgid_5'] < 4,'coef_d'] = -1.407847328
#Gen4i A1
dfAIw.loc[dfAIw['mfgid_5'] >= 4,'coef_a'] = -0.015470889
dfAIw.loc[dfAIw['mfgid_5'] >= 4,'coef_b'] = 1.684161818
dfAIw.loc[dfAIw['mfgid_5'] >= 4,'coef_c'] = 1.335027916
dfAIw.loc[dfAIw['mfgid_5'] >= 4,'coef_d'] = 0.387637386

dfAIw.loc[:,'ipk'] = ((dfAIw.loc[:,'coef_a'].values * dfAIw.loc[:,'ka'].values + dfAIw.loc[:,'coef_b'].values) * dfAIw.loc[:,'iw'].values +
                      (dfAIw.loc[:,'coef_c'].values * dfAIw.loc[:,'ka'].values + dfAIw.loc[:,'coef_d'].values)
                     )
dfAIw.to_pickle('dfAIw.pkl')
dfAIw

current sql is select hddsn, enddate, pfcode, testcode, mfgid, mfgid_5, hddtrial, qualifier, lhd, phd, readerselection,                         cmdexecutiontimeinmilliseconds as time, band, cylinder, meastemp,                         writecurrent as iw, kickamplitude as ka, kickduration as kd, bestser, readoffset, enddt,                         row_number() over ( partition by hddsn, testcode, qualifier, lhd, band order by enddate desc) as row_num from ghl2.ccb_ci_aiw_summary where product='pdq' and testcode in ('PDQC3EKM', 'PDQX073M') and enddt between '20201204' and '20210126' and hddtrial in ('M1TJ', 'M1U2', 'M1U3', '0000') and hddsn in ('2FA0A3AD', '2FA0A43D', '2FA0A4BD', '2FA0A59N', '2FA0A5NN', '2FA0A5VN', '2FA0A2WD', '2FA0A2LD', '2FA0A62N', '2FA09ZSF', '2FA0A63N', '2FA0A6HN', '2FA0A3ZD', '2FA0A37D', '2FA0A2XD', '2FA0A1ZD', '2FA0A4SN', '2FA0A4DD', '2FA0A4HD', '2FA0A3RD', '2FA0A6SN', '2FA0A20D', '2FA0A2RD', '2FA0A17F', '2FA0A5YN', '2FA0A6RN', '2FA0A36D', '2FA0A6GN', '2FA0A3HD', '2F

,hddsn,enddate,pfcode,testcode,mfgid,mfgid_5,hddtrial,qualifier,lhd,phd,...,kd,bestser,readoffset,enddt,row_num,coef_a,coef_b,coef_c,coef_d,ipk
0,2FA09ZHF,20201205030003,0000,PDQX073M,KKBF04,4,M1TJ,11N0,13,13,...,13,-1.87,-1.54,20201205,1,-0.015471,1.684162,1.335028,0.387637,66.958226
1,2FA0A17F,20201205020359,0000,PDQX073M,KKBF04,4,M1TJ,10N0,14,14,...,8,-2.99,0.10,20201205,1,-0.015471,1.684162,1.335028,0.387637,81.524123
2,2FA0A18F,20201205013233,0000,PDQX073M,KKBF04,4,M1TJ,30N0,1,1,...,11,-2.11,0.32,20201205,1,-0.015471,1.684162,1.335028,0.387637,69.188401
3,2FA0A20D,20210126092715,0000,PDQC3EKM,KQBF04,4,0000,10N0,12,12,...,7,-3.23,-0.45,20210126,1,-0.015471,1.684162,1.335028,0.387637,85.359316
4,2FA0A20D,20201205001150,0000,PDQX073M,KQBF04,4,M1U2,30N0,2,2,...,6,-2.17,-1.06,20201205,1,-0.015471,1.684162,1.335028,0.387637,91.680616
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37489,2FA0A4SN,20201205001915,0000,PDQX073M,KQBF04,4,M1U3,10N0,11,11,...,11,-2.93,-0.41,20201205,1,-0.015471,1.684162,1.335028,0.387637,74.426826
37490,2FA0A5BN,20210126105827,0000,PDQC3EKM,KQBF04,4,0000,10N0,14,14,...,13,-3.06,-0.53,20210126,1,-0.015471,1.684162,1.335028,0.387637,68.330517
37491,2FA0A5YN,20201205002246,0000,PDQX073M,KQBF04,4,M1U3,30N0,3,3,...,8,-2.21,-0.24,20201205,1,-0.015471,1.684162,1.335028,0.387637,82.348612
37492,2FA0A63N,20201204232820,0000,PDQX073M,KQBF04,4,M1U3,10N0,16,16,...,6,-3.22,-0.28,20201204,1,-0.015471,1.684162,1.335028,0.387637,90.918011


In [27]:
#AIw Profile
inputParamGhl2AIw = [snlist_csv, 
                      listTestcode,
                      listEnddt, 
                      listHddTrial,
                      ['6400'],
                      ['10N0', '11N0', '30N0', '20N0', '40N0']
                     ]
dfAIw = get_ghl2_table('ccb_ci_aiw_prim_comb', 
                        inputParamGhl2AIw, 
                       'hddsn, enddate, pfcode, testcode, mfgid, mfgid_5, hddtrial, qualifier, lhd, phd, readerselection, \
                        band, index, writecurlist as iw, kickamp as ka, peakiw as ipk0, servswrparm as ser, \
                        row_number() over ( partition by hddsn, qualifier, lhd, band, peakiw, testcode order by enddate desc) as row_num'
                       )
dfAIw.loc[:,'mfgid_5'] = dfAIw['mfgid_5'].values.astype('int')

#Gen4
dfAIw.loc[dfAIw['mfgid_5'] < 4,'coef_a'] = -0.013954886
dfAIw.loc[dfAIw['mfgid_5'] < 4,'coef_b'] = 1.128161219
dfAIw.loc[dfAIw['mfgid_5'] < 4,'coef_c'] = 1.973010793
dfAIw.loc[dfAIw['mfgid_5'] < 4,'coef_d'] = -1.407847328
#Gen4i A1
dfAIw.loc[dfAIw['mfgid_5'] >= 4,'coef_a'] = -0.015470889
dfAIw.loc[dfAIw['mfgid_5'] >= 4,'coef_b'] = 1.684161818
dfAIw.loc[dfAIw['mfgid_5'] >= 4,'coef_c'] = 1.335027916
dfAIw.loc[dfAIw['mfgid_5'] >= 4,'coef_d'] = 0.387637386

dfAIw.loc[:,'ipk'] = ((dfAIw.loc[:,'coef_a'].values * dfAIw.loc[:,'ka'].values + dfAIw.loc[:,'coef_b'].values) * dfAIw.loc[:,'iw'].values +
                      (dfAIw.loc[:,'coef_c'].values * dfAIw.loc[:,'ka'].values + dfAIw.loc[:,'coef_d'].values)
                     )
dfAIw.to_pickle('dfAIwProfile.pkl')
dfAIw

current sql is select hddsn, enddate, pfcode, testcode, mfgid, mfgid_5, hddtrial, qualifier, lhd, phd, readerselection,                         band, index, writecurlist as iw, kickamp as ka, peakiw as ipk0, servswrparm as ser,                         row_number() over ( partition by hddsn, qualifier, lhd, band, peakiw, testcode order by enddate desc) as row_num from ghl2.ccb_ci_aiw_prim_comb where product='pdq' and testcode in ('PDQC3EKM', 'PDQX073M') and enddt between '20201204' and '20210126' and hddtrial in ('M1TJ', 'M1U2', 'M1U3', '0000') and hddsn in ('2FA0A3AD', '2FA0A43D', '2FA0A4BD', '2FA0A59N', '2FA0A5NN', '2FA0A5VN', '2FA0A2WD', '2FA0A2LD', '2FA0A62N', '2FA09ZSF', '2FA0A63N', '2FA0A6HN', '2FA0A3ZD', '2FA0A37D', '2FA0A2XD', '2FA0A1ZD', '2FA0A4SN', '2FA0A4DD', '2FA0A4HD', '2FA0A3RD', '2FA0A6SN', '2FA0A20D', '2FA0A2RD', '2FA0A17F', '2FA0A5YN', '2FA0A6RN', '2FA0A36D', '2FA0A6GN', '2FA0A3HD', '2FA0A4PN', '2FA0A32D', '2FA0A4MN', '2FA0A6EN', '2FA0A3ND', '2FA0A0VF', '2FA0A3LD', '2FA

,hddsn,enddate,pfcode,testcode,mfgid,mfgid_5,hddtrial,qualifier,lhd,phd,...,iw,ka,ipk0,ser,row_num,coef_a,coef_b,coef_c,coef_d,ipk
0,2FA09ZHF,20201205030003,0000,PDQX073M,KKBF04,4,M1TJ,10N0,1,1,...,36,36,92,-2.36,1,-0.015471,1.684162,1.335028,0.387637,89.028196
1,2FA09ZHF,20201205030003,0000,PDQX073M,KKBF04,4,M1TJ,10N0,9,9,...,14,10,32,-0.07,1,-0.015471,1.684162,1.335028,0.387637,35.150258
2,2FA09ZHF,20201205030003,0000,PDQX073M,KKBF04,4,M1TJ,10N0,10,10,...,18,14,43,-1.19,1,-0.015471,1.684162,1.335028,0.387637,45.494277
3,2FA09ZHF,20201205030003,0000,PDQX073M,KKBF04,4,M1TJ,11N0,5,5,...,18,14,43,-0.32,1,-0.015471,1.684162,1.335028,0.387637,45.494277
4,2FA09ZHF,20201205030003,0000,PDQX073M,KKBF04,4,M1TJ,11N0,9,9,...,22,18,53,-0.86,1,-0.015471,1.684162,1.335028,0.387637,55.343228
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
801769,2FA0A6XN,20210126100917,0000,PDQC3EKM,KQBF04,4,0000,30N0,8,8,...,8,5,20,0.00,1,-0.015471,1.684162,1.335028,0.387637,19.917236
801770,2FA0A6XN,20210126100917,0000,PDQC3EKM,KQBF04,4,0000,30N0,10,10,...,20,16,50,-0.94,1,-0.015471,1.684162,1.335028,0.387637,50.480636
801771,2FA0A6XN,20201205001910,0000,PDQX073M,KQBF04,4,M1U3,30N0,11,11,...,4,2,7,0.00,1,-0.015471,1.684162,1.335028,0.387637,9.670573
801772,2FA0A6XN,20210126100917,0000,PDQC3EKM,KQBF04,4,0000,30N0,12,12,...,14,10,35,-0.27,1,-0.015471,1.684162,1.335028,0.387637,35.150258


# ATI

In [23]:
#DATI Nmax
inputParamGhl2Dati = [snlist_csv, 
                     listTestcode,
                     listEnddt, 
                     listHddTrial,
                     ['6400', '6600'],
                     ['3000', '9000', '4360', 'L360']
                    ]
dfNmaxTemp = get_ghl2_table('ccb_ci_ati_prim_bybandwrnum', 
                            inputParamGhl2Dati, 
                            'hddsn, enddt, mfgid, hddtrial, testcode, qualifier, procid, lhd, phd, band, nmax50byband, \
                             row_number() over (partition by hddsn, testcode, qualifier, lhd, band order by enddate desc) as row_num')
dfNmaxTemp = pd.read_pickle('df_ccb_ci_ati_prim_bybandwrnum.pkl')
dfNmaxTemp.loc[:,'nmax50byband'] = dfNmaxTemp.loc[:,'nmax50byband'].values.astype(np.int64)
dfNmaxTemp.to_pickle('dfNmaxTemp.pkl')
dfNmaxTemp

current sql is select hddsn, enddt, mfgid, hddtrial, testcode, qualifier, procid, lhd, phd, band, nmax50byband,                              row_number() over (partition by hddsn, testcode, qualifier, lhd, band order by enddate desc) as row_num from ghl2.ccb_ci_ati_prim_bybandwrnum where product='pdq' and testcode in ('PDQC3EKM', 'PDQX073M') and enddt between '20201204' and '20210204' and hddtrial in ('M1TJ', 'M1U2', 'M1U3', '0000') and hddsn in ('2FA0A3AD', '2FA0A43D', '2FA0A4BD', '2FA0A59N', '2FA0A5NN', '2FA0A5VN', '2FA0A2WD', '2FA0A2LD', '2FA0A62N', '2FA09ZSF', '2FA0A63N', '2FA0A6HN', '2FA0A3ZD', '2FA0A37D', '2FA0A2XD', '2FA0A1ZD', '2FA0A4SN', '2FA0A4DD', '2FA0A4HD', '2FA0A3RD', '2FA0A6SN', '2FA0A20D', '2FA0A2RD', '2FA0A17F', '2FA0A5YN', '2FA0A6RN', '2FA0A36D', '2FA0A6GN', '2FA0A3HD', '2FA0A4PN', '2FA0A32D', '2FA0A4MN', '2FA0A6EN', '2FA0A3ND', '2FA0A0VF', '2FA0A3LD', '2FA0A23D', '2FA0A2TD', '2FA0A40D', '2FA0A6XN', '2FA0A3TD', '2FA0A2KD', '2FA0A27D', '2FA0A4ZN', '2FA0A5BN', '2FA0A5XN

,hddsn,enddt,mfgid,hddtrial,testcode,qualifier,procid,lhd,phd,band,nmax50byband,row_num
0,2FA09ZHF,20210126,KKBF04,0000,PDQC3EKM,3000,6400,6,6,55,1495,1
1,2FA09ZHF,20210126,KKBF04,0000,PDQC3EKM,3000,6400,17,17,54,3063,1
2,2FA09ZHF,20210204,KKBF04,0000,PDQC3EKM,9000,6600,3,3,32,483,1
3,2FA09ZHF,20201205,KKBF04,M1TJ,PDQX073M,3000,6400,3,3,33,223,1
4,2FA09ZHF,20201205,KKBF04,M1TJ,PDQX073M,3000,6400,3,3,59,942,1
...,...,...,...,...,...,...,...,...,...,...,...,...
230395,2FA0A6XN,20201205,KQBF04,M1U3,PDQX073M,3000,6400,6,6,56,375,1
230396,2FA0A6XN,20201205,KQBF04,M1U3,PDQX073M,3000,6400,16,16,49,134,1
230397,2FA0A6XN,20201214,KQBF04,M1U3,PDQX073M,9000,6600,1,1,17,76,1
230398,2FA0A6XN,20201214,KQBF04,M1U3,PDQX073M,9000,6600,8,8,16,78,1


In [24]:
#DATI RV
inputParamGhl2AtiRv = [snlist_csv, 
                       listTestcode,
                       listEnddt, 
                       listHddTrial,
                       ['6400', '6600'],
                       ['35N0', '3000', '95H0', '9000', '4360', 'L360']
                      ]

dfAtiRv = get_ghl2_table('ccb_ci_dati_theta', 
                         inputParamGhl2AtiRv, 
                         'hddsn, enddate, pfcode, testcode, testcodec, mfgid, hddtrial, cmdname, qualifier, lhd, phd, subqualifier, readerselection, tempzone, zone, \
                          maxrvsumheadalltemp, avgrvsumheadalltemp, avgrvsumdrivealltemp, avgrvsumzonealltemp, rvsum, spaidentifier, rvsumminushalf4smr, rvsumplushalf4smr, \
                          worstavgrvsumzonewithmaxnlxmulti, worstzonewithmaxnlxmulti, worstavgrvsumzonewithminnlxmulti, worstzonewithminnlxmulti, worstavgrvsumzonewithavenlxmulti, \
                          worstzonewithavenlxmulti, product, procid, enddt, \
                          row_number() over ( partition by hddsn, testcode, qualifier, lhd, tempzone, zone order by enddate desc) as row_num')
#dfAtiRv = pd.read_pickle('df_ccb_ci_dati_theta.pkl')
dfAtiRv

current sql is select hddsn, enddate, pfcode, testcode, testcodec, mfgid, hddtrial, cmdname, qualifier, lhd, phd, subqualifier, readerselection, tempzone, zone,                           maxrvsumheadalltemp, avgrvsumheadalltemp, avgrvsumdrivealltemp, avgrvsumzonealltemp, rvsum, spaidentifier, rvsumminushalf4smr, rvsumplushalf4smr,                           worstavgrvsumzonewithmaxnlxmulti, worstzonewithmaxnlxmulti, worstavgrvsumzonewithminnlxmulti, worstzonewithminnlxmulti, worstavgrvsumzonewithavenlxmulti,                           worstzonewithavenlxmulti, product, procid, enddt,                           row_number() over ( partition by hddsn, testcode, qualifier, lhd, tempzone, zone order by enddate desc) as row_num from ghl2.ccb_ci_dati_theta where product='pdq' and testcode in ('PDQC3EKM', 'PDQX073M') and enddt between '20201204' and '20210204' and hddtrial in ('M1TJ', 'M1U2', 'M1U3', '0000') and hddsn in ('2FA0A3AD', '2FA0A43D', '2FA0A4BD', '2FA0A59N', '2FA0A5NN', '2FA0A5VN', '2

,hddsn,enddate,pfcode,testcode,testcodec,mfgid,hddtrial,cmdname,qualifier,lhd,...,worstavgrvsumzonewithmaxnlxmulti,worstzonewithmaxnlxmulti,worstavgrvsumzonewithminnlxmulti,worstzonewithminnlxmulti,worstavgrvsumzonewithavenlxmulti,worstzonewithavenlxmulti,product,procid,enddt,row_num
0,2FA09ZHF,20210126120110,0000,PDQC3EKM,PDQC3EK,KKBF04,0000,CAti,3000,0,...,0,0,0,0,0,0,pdq,6400,20210126,1
1,2FA09ZHF,20210126120110,0000,PDQC3EKM,PDQC3EK,KKBF04,0000,CAti,3000,3,...,0,0,0,0,0,0,pdq,6400,20210126,1
2,2FA09ZHF,20210126120110,0000,PDQC3EKM,PDQC3EK,KKBF04,0000,CAti,3000,6,...,0,0,0,0,0,0,pdq,6400,20210126,1
3,2FA09ZHF,20210126120110,0000,PDQC3EKM,PDQC3EK,KKBF04,0000,CAti,3000,8,...,0,0,0,0,0,0,pdq,6400,20210126,1
4,2FA09ZHF,20210126120110,0000,PDQC3EKM,PDQC3EK,KKBF04,0000,CAti,3000,15,...,0,0,0,0,0,0,pdq,6400,20210126,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
993595,2FA0A6HN,20201214073919,TRW1,PDQX073M,PDQX073,KQBF04,M1U3,CAti,9000,4,...,0,0,0,0,0,0,pdq,6600,20201214,1
993596,2FA0A6HN,20201214073919,TRW1,PDQX073M,PDQX073,KQBF04,M1U3,CAti,9000,6,...,0,0,0,0,0,0,pdq,6600,20201214,1
993597,2FA0A6HN,20201214073919,TRW1,PDQX073M,PDQX073,KQBF04,M1U3,CAti,9000,14,...,0,0,0,0,0,0,pdq,6600,20201214,1
993598,2FA0A6HN,20201214073919,TRW1,PDQX073M,PDQX073,KQBF04,M1U3,CAti,9000,17,...,0,0,0,0,0,0,pdq,6600,20201214,1


In [40]:
dfAtiRv.testcode.value_counts()

PDQX073M    506736
PDQC3EKM    213624
Name: testcode, dtype: int64

In [25]:
#JINX RV
inputParamGhl2AtiRv = [snlist_csv, 
                       listTestcode,
                       listEnddt, 
                       listHddTrial,
                       ['6600'],
                       ['9050']
                      ]

dfJinxRv = get_ghl2_table('ccb_ci_ati_mcsb_ns', 
                         inputParamGhl2AtiRv, 
                         'hddsn, enddate, pfcode, testcode, testcodec, mfgid, hddtrial, cmdname, qualifier, lhd, phd, maxrvcheck4nonlinearxcor_0, maxrvcheck4nonlinearxcor_1, \
                          row_number() over ( partition by hddsn, testcode, qualifier, lhd order by enddate desc) as row_num')
dfJinxRv

current sql is select hddsn, enddate, pfcode, testcode, testcodec, mfgid, hddtrial, cmdname, qualifier, lhd, phd, maxrvcheck4nonlinearxcor_0, maxrvcheck4nonlinearxcor_1,                           row_number() over ( partition by hddsn, testcode, qualifier, lhd order by enddate desc) as row_num from ghl2.ccb_ci_ati_mcsb_ns where product='pdq' and testcode in ('PDQC3EKM', 'PDQX073M') and enddt between '20201204' and '20210204' and hddtrial in ('M1TJ', 'M1U2', 'M1U3', '0000') and hddsn in ('2FA0A3AD', '2FA0A43D', '2FA0A4BD', '2FA0A59N', '2FA0A5NN', '2FA0A5VN', '2FA0A2WD', '2FA0A2LD', '2FA0A62N', '2FA09ZSF', '2FA0A63N', '2FA0A6HN', '2FA0A3ZD', '2FA0A37D', '2FA0A2XD', '2FA0A1ZD', '2FA0A4SN', '2FA0A4DD', '2FA0A4HD', '2FA0A3RD', '2FA0A6SN', '2FA0A20D', '2FA0A2RD', '2FA0A17F', '2FA0A5YN', '2FA0A6RN', '2FA0A36D', '2FA0A6GN', '2FA0A3HD', '2FA0A4PN', '2FA0A32D', '2FA0A4MN', '2FA0A6EN', '2FA0A3ND', '2FA0A0VF', '2FA0A3LD', '2FA0A23D', '2FA0A2TD', '2FA0A40D', '2FA0A6XN', '2FA0A3TD', '2FA0A2KD', '2FA

,hddsn,enddate,pfcode,testcode,testcodec,mfgid,hddtrial,cmdname,qualifier,lhd,phd,maxrvcheck4nonlinearxcor_0,maxrvcheck4nonlinearxcor_1,row_num
0,2FA0A6XN,20210203232034,0000,PDQC3EKM,PDQC3EK,KQBF04,0000,CAti,9050,2,2,998,15000,1
1,2FA0A4BD,20201214090023,TRJ1,PDQX073M,PDQX073,KQBF04,M1U2,CAti,9050,6,6,5922,15000,1
2,2FA0A43D,20201214092656,TRJ1,PDQX073M,PDQX073,KQBF04,M1U2,CAti,9050,14,14,4112,15000,1
3,2FA0A6EN,20210204010204,0000,PDQC3EKM,PDQC3EK,KQBF04,0000,CAti,9050,7,7,1198,15000,1
4,2FA0A35D,20210204011014,0000,PDQC3EKM,PDQC3EK,KQBF04,0000,CAti,9050,1,1,1120,15000,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1795,2FA09ZHF,20201214160858,TRJ1,PDQX073M,PDQX073,KKBF04,M1TJ,CAti,9050,3,3,5456,15000,1
1796,2FA0A1AF,20201214134211,TRJ1,PDQX073M,PDQX073,KKBF04,M1TJ,CAti,9050,12,12,6741,15000,1
1797,2FA0A4HD,20201214053845,TRJ1,PDQX073M,PDQX073,KQBF04,M1U2,CAti,9050,9,9,5278,15000,1
1798,2FA0A3LD,20210203233451,0000,PDQC3EKM,PDQC3EK,KQBF04,0000,CAti,9050,0,0,1989,15000,1


# ASB (SMR)

In [30]:
#CASB
inputParamGhl2Asb = [snlist_csv, 
                     listTestcode,
                     listEnddt, 
                     listHddTrial,
                     ['6400'],
                     ['2300']
                    ]
dfAsb = get_ghl2_table('ccb_ci_casb', inputParamGhl2Asb, 
                       'hddsn, enddate, lhd, phd, mfgid, testcode, maxbandod2id, \
                        row_number() over (partition by hddsn, testcode, lhd, band order by enddate desc) as row_num', 
                       'band=0')
dfAsb = pd.read_pickle('df_ccb_ci_casb.pkl').query("row_num==1")
dfAsb

current sql is select hddsn, enddate, lhd, phd, mfgid, testcode, maxbandod2id,                         row_number() over (partition by hddsn, testcode, lhd, band order by enddate desc) as row_num from ghl2.ccb_ci_casb where product='pdq' and testcode in ('PDQX083M') and enddt between '20201228' and '20210107' and hddsn in ('2FA00EPA', '2FA01HYA', '2FA026TA', '2FA00E8A', '2FA01K0A', '2FA026YA', '2FA00EWA', '2FA00ESA', '2FA00EYA', '2FA0093A') and procid in ('6400') and qualifier in ('2300') and pfcode not in ('9999', '99V6', '99V7') and band=0 
(64, 8)


,hddsn,enddate,lhd,phd,mfgid,testcode,maxbandod2id,row_num
0,2FA026TA,20201228075939,2,8,KJ1E03,PDQX083M,29,1
1,2FA01HYA,20201228072658,2,8,KJBE03,PDQX083M,31,1
2,2FA00EYA,20201228083039,0,0,KJ1E03,PDQX083M,48,1
3,2FA01HYA,20201228072658,4,10,KJBE03,PDQX083M,35,1
4,2FA00E8A,20201228075844,6,16,KJ1E03,PDQX083M,32,1
...,...,...,...,...,...,...,...,...
59,2FA00EPA,20201228063812,0,0,KJ1E03,PDQX083M,29,1
60,2FA00E8A,20201228075844,7,17,KJ1E03,PDQX083M,42,1
61,2FA00EPA,20201228063812,1,1,KJ1E03,PDQX083M,32,1
62,2FA026TA,20201228075939,1,1,KJ1E03,PDQX083M,61,1


# UFO

In [26]:
# UFO
inputParamGhl2Ufo = [snlist_csv, 
                     listTestcode,
                     listEnddt, 
                     listHddTrial,
                     ['6400'],
                     ['1000', '2000']
                    ]
dfUfo = get_ghl2_table('ccb_ci_ufo', inputParamGhl2Ufo, 
                       'hddsnenddate, hddsn, enddate, drivemodel, pfcode, testcode, mfgid, hddtrial, aetype, hdctype, cmdname, \
                       qualifier, readerselection, cmdexecutiontimeinmilliseconds, mcsbarraymaxnumheads, mcsbarraymaxnumbands, \
                       mcsbarrayodmdidbands, logicalheadtestcompletemap, physicalheadfailcriteriamap, lastlogicalheadtested, lastbandtested, \
                       lasttracktested, lastsubcmdtested, finalusedservotrack, finalstrokepctx100, finalstrokepercent, finaldefectupliftpctx100, \
                       finalmediacacheupliftpctx100, finaltrackskewadjustment, finaltpirelaxationpercentx100, finaltpirelaxationpercent, \
                       finalwaterfallselection, finalblocksizemap, finalfitmetricvalue, finalmaxlogicalheadnum, finalmodelnumber, finalphysicalheadmap, \
                       finalsustainedmbytepersec, totaldrivelbarequired, finaldrivetotallba, calcsteps, accdrivetotallba, accdrivetotallbanoati, \
                       maxchanfreqmhz, accdrivetotallbaformat, finalseqperfvarpctx100, accdefaultupliftcalcreqcap, product, procid, enddt, \
                       row_number() over (partition by hddsn, testcode, qualifier order by enddate desc) as row_num')
#dfUfo.loc[dfUfo.qualifier.str.startswith('1'), 'targetLba'] = tgtLbaCmr
#dfUfo.loc[dfUfo.qualifier.str.startswith('2'), 'targetLba'] = tgtLbaSmr
#dfUfo.loc[:,'iacc_pct'] = dfUfo.loc[:,'iacc_pct'].values / tgtLbaCmr
dfUfo = pd.read_pickle('df_ccb_ci_ufo.pkl')

current sql is select hddsnenddate, hddsn, enddate, drivemodel, pfcode, testcode, mfgid, hddtrial, aetype, hdctype, cmdname,                        qualifier, readerselection, cmdexecutiontimeinmilliseconds, mcsbarraymaxnumheads, mcsbarraymaxnumbands,                        mcsbarrayodmdidbands, logicalheadtestcompletemap, physicalheadfailcriteriamap, lastlogicalheadtested, lastbandtested,                        lasttracktested, lastsubcmdtested, finalusedservotrack, finalstrokepctx100, finalstrokepercent, finaldefectupliftpctx100,                        finalmediacacheupliftpctx100, finaltrackskewadjustment, finaltpirelaxationpercentx100, finaltpirelaxationpercent,                        finalwaterfallselection, finalblocksizemap, finalfitmetricvalue, finalmaxlogicalheadnum, finalmodelnumber, finalphysicalheadmap,                        finalsustainedmbytepersec, totaldrivelbarequired, finaldrivetotallba, calcsteps, accdrivetotallba, accdrivetotallbanoati,                        maxch

In [27]:
# UFO hd array
inputParamGhl2UfoHdArry = [snlist_csv, 
                           listTestcode,
                           listEnddt, 
                           listHddTrial,
                           ['6400'],
                           ['1000', '2000']
                    ]
dfUfoHdArry = get_ghl2_table('ccb_ci_ufo_hd_array', inputParamGhl2UfoHdArry, 
                             'hddsn, enddate, pfcode, testcode, mfgid, hddtrial, qualifier, lhd, readerselection, \
                             finalheadtotallba, accheadtotallba, accmetricvalue, accheadtotallbanoati, \
                             accheadtotallbaformat, \
                             row_number() over (partition by hddsn, lhd, testcode, qualifier order by enddate desc) as row_num',
                            )
dfUfoHdArry.loc[dfUfoHdArry.qualifier.str.startswith('1'), 'targetLba'] = tgtLbaCmr
dfUfoHdArry.loc[dfUfoHdArry.qualifier.str.startswith('2'), 'targetLba'] = tgtLbaSmr
dfUfoHdArry.loc[:,'iacc_pct'] = 100*dfUfoHdArry.loc[:,'accheadtotallbanoati'].values  / dfUfoHdArry.loc[:, 'targetLba'].values
dfUfoHdArry.loc[:,'pacc_pct'] = 100*dfUfoHdArry.loc[:,'accheadtotallbaformat'].values / dfUfoHdArry.loc[:, 'targetLba'].values
dfUfoHdArry.to_pickle('dfUfoHdArry.pkl')
dfUfoHdArry

current sql is select hddsn, enddate, pfcode, testcode, mfgid, hddtrial, qualifier, lhd, readerselection,                              finalheadtotallba, accheadtotallba, accmetricvalue, accheadtotallbanoati,                              accheadtotallbaformat,                              row_number() over (partition by hddsn, lhd, testcode, qualifier order by enddate desc) as row_num from ghl2.ccb_ci_ufo_hd_array where product='pdq' and testcode in ('PDQC3EKM', 'PDQX073M') and enddt between '20201204' and '20210204' and hddtrial in ('M1TJ', 'M1U2', 'M1U3', '0000') and hddsn in ('2FA0A3AD', '2FA0A43D', '2FA0A4BD', '2FA0A59N', '2FA0A5NN', '2FA0A5VN', '2FA0A2WD', '2FA0A2LD', '2FA0A62N', '2FA09ZSF', '2FA0A63N', '2FA0A6HN', '2FA0A3ZD', '2FA0A37D', '2FA0A2XD', '2FA0A1ZD', '2FA0A4SN', '2FA0A4DD', '2FA0A4HD', '2FA0A3RD', '2FA0A6SN', '2FA0A20D', '2FA0A2RD', '2FA0A17F', '2FA0A5YN', '2FA0A6RN', '2FA0A36D', '2FA0A6GN', '2FA0A3HD', '2FA0A4PN', '2FA0A32D', '2FA0A4MN', '2FA0A6EN', '2FA0A3ND', '2FA0A

,hddsn,enddate,pfcode,testcode,mfgid,hddtrial,qualifier,lhd,readerselection,finalheadtotallba,accheadtotallba,accmetricvalue,accheadtotallbanoati,accheadtotallbaformat,row_num,targetLba,iacc_pct,pacc_pct
0,2FA0A36D,20201204233500,0000,PDQX073M,KQBF04,M1U2,1000,8,3951,257280486,265612493.0,-105.0,268088837,265612493,1,273906721.0,97.875962,96.971879
1,2FA0A5VN,20201205003220,0000,PDQX073M,KQBF04,M1U3,1000,9,3951,264083747,271283308.0,-105.0,272646992,271283308,1,273906721.0,99.540088,99.042224
2,2FA0A3AD,20201204235332,0000,PDQX073M,KQBF04,M1U2,1000,4,3951,254869696,262123099.0,-105.0,265726173,262123099,1,273906721.0,97.013382,95.697943
3,2FA0A4SN,20201205001915,0000,PDQX073M,KQBF04,M1U3,1000,8,3951,266296423,271560180.0,-105.0,273583935,271560180,1,273906721.0,99.882155,99.143307
4,2FA0A20D,20201205001150,0000,PDQX073M,KQBF04,M1U2,1000,12,3951,261875559,271099940.0,-105.0,272652809,271099940,1,273906721.0,99.542212,98.975279
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1813,2FA0A18F,20210126111302,0000,PDQC3EKM,KKBF04,0000,1000,0,4079,264168561,NaN,NaN,276379723,276190175,1,273906721.0,100.902863,100.833661
1814,2FA0A4SN,20210126112009,0000,PDQC3EKM,KQBF04,0000,1000,10,3951,271483157,NaN,NaN,282324043,282324043,1,273906721.0,103.073062,103.073062
1815,2FA0A5YN,20210126104626,0000,PDQC3EKM,KQBF04,0000,1000,13,3951,263631710,NaN,NaN,273942979,273406369,1,273906721.0,100.013237,99.817328
1816,2FA0A6XN,20201205001910,0000,PDQX073M,KQBF04,M1U3,1000,4,3951,271603869,281649269.0,-105.0,282443240,281649269,1,273906721.0,103.116579,102.826710


In [33]:
pd.pivot_table(dfUfoHdArry.query("qualifier=='1000' and lhd==0"), index=['hddsn', 'mfgid'], columns=['testcode'], values=['accheadtotallbanoati'], 
               aggfunc=np.mean)

accheadtotallbanoati             
testcode                    PDQC3EKM     PDQX073M
hddsn    mfgid                                   
2FA09ZHF KKBF04          266200195.0  269828823.0
2FA09ZSF KKBF04          254779409.0  260247202.0
2FA0A03F KKBF04          257064783.0  260148954.0
2FA0A0VF KKBF04          264759139.0  265590087.0
2FA0A17F KKBF04          265634965.0  268766006.0
2FA0A18F KKBF04          276379723.0  277244590.0
2FA0A1AF KKBF04          271790209.0  273785628.0
2FA0A1ZD KQBF04          274986631.0  276979473.0
2FA0A20D KQBF04          273080676.0  276188436.0
2FA0A23D KQBF04          270451178.0  273767226.0
2FA0A27D KQBF04          269743924.0  273984338.0
2FA0A2KD KQBF04          273367984.0  275891726.0
2FA0A2LD KQBF04          276117810.0  281525346.0
2FA0A2RD KQBF04          269113068.0  272421308.0
2FA0A2TD KQBF04          272420854.0  276189691.0
2FA0A2WD KQBF04          275373261.0  275645782.0
2FA0A2XD KQBF04          268162303.0  272402388.0
2FA0A32D KQBF04          281635024.0  283570352.0
2FA0A35D KQBF04          276920186.0  279638418.0
2FA0A36D KQBF04          268181478.0  272842563.0
2FA0A37D KQBF04          266726506.0  270300809.0
2FA0A3AD KQBF04          262975447.0  266293475.0
2FA0A3HD KQBF04          279264664.0  276113482.0
2FA0A3LD KQBF04          267874663.0  272610961.0
2FA0A3ND KQBF04          277312335.0  280767261.0
2FA0A3RD KQBF04          280753216.0  283688302.0
2FA0A3TD KQBF04          278101427.0  281664209.0
2FA0A3ZD KQBF04          276928526.0  282546796.0
2FA0A40D KQBF04          268561310.0  272963165.0
2FA0A43D KQBF04          272080271.0  277758204.0
2FA0A4BD KQBF04          269531748.0  272630526.0
2FA0A4DD KQBF04          272873122.0  274922955.0
2FA0A4HD KQBF04          262117063.0  266209710.0
2FA0A4MN KQBF04          269249350.0  274348190.0
2FA0A4PN KQBF04          272642007.0  276042104.0
2FA0A4SN KQBF04          276517984.0  281740765.0
2FA0A4ZN KQBF04          275595296.0  278942516.0
2FA0A59N KQBF04          264680198.0  269369128.0
2FA0A5BN KQBF04          271331093.0  274242577.0
2FA0A5NN KQBF04          267268582.0  269653730.0
2FA0A5VN KQBF04                  NaN  277841411.0
2FA0A5XN KQBF04          263660700.0  266643534.0
2FA0A5YN KQBF04          271931105.0  276502838.0
2FA0A62N KQBF04          268076006.0  273120737.0
2FA0A63N KQBF04          263203956.0  279193435.0
2FA0A6EN KQBF04          281880801.0  285614765.0
2FA0A6GN KQBF04          269128791.0  274554002.0
2FA0A6HN KQBF04          267516301.0  274437926.0
2FA0A6RN KQBF04          276025647.0  276189004.0
2FA0A6SN KQBF04          277921303.0  280016155.0
2FA0A6XN KQBF04          271060519.0  276106437.0

In [29]:
# UFO SER, TPC
inputParamGhl2UfoSubCmd1 = [snlist_csv, 
                            listTestcode,
                            listEnddt, 
                            listHddTrial,
                            ['6400'],
                            ['1000', '2000']
                           ]
dfUfoSubCmd1 = get_ghl2_table('ccb_ci_ufo_scmd1', inputParamGhl2UfoSubCmd1, 
                              '* ')
dfUfoSubCmd1 = pd.read_pickle('df_ccb_ci_ufo_scmd1.pkl')

current sql is select *  from ghl2.ccb_ci_ufo_scmd1 where product='pdq' and testcode in ('PDQC3EKM', 'PDQX073M') and enddt between '20201204' and '20210204' and hddtrial in ('M1TJ', 'M1U2', 'M1U3', '0000') and hddsn in ('2FA0A3AD', '2FA0A43D', '2FA0A4BD', '2FA0A59N', '2FA0A5NN', '2FA0A5VN', '2FA0A2WD', '2FA0A2LD', '2FA0A62N', '2FA09ZSF', '2FA0A63N', '2FA0A6HN', '2FA0A3ZD', '2FA0A37D', '2FA0A2XD', '2FA0A1ZD', '2FA0A4SN', '2FA0A4DD', '2FA0A4HD', '2FA0A3RD', '2FA0A6SN', '2FA0A20D', '2FA0A2RD', '2FA0A17F', '2FA0A5YN', '2FA0A6RN', '2FA0A36D', '2FA0A6GN', '2FA0A3HD', '2FA0A4PN', '2FA0A32D', '2FA0A4MN', '2FA0A6EN', '2FA0A3ND', '2FA0A0VF', '2FA0A3LD', '2FA0A23D', '2FA0A2TD', '2FA0A40D', '2FA0A6XN', '2FA0A3TD', '2FA0A2KD', '2FA0A27D', '2FA0A4ZN', '2FA0A5BN', '2FA0A5XN', '2FA0A35D', '2FA0A18F', '2FA0A03F', '2FA09ZHF', '2FA0A1AF') and procid in ('6400') and qualifier in ('1000', '2000') and pfcode not in ('9999', '99V6', '99V7') 
(177257, 62)


In [30]:
# UFO Sqz Margin, otrc, track pitch
inputParamGhl2UfoSubCmd7 = [snlist_csv, 
                            listTestcode,
                            listEnddt, 
                            listHddTrial,
                            ['6400', '6600'],
                            ['3000', '9000', '4000', 'K000']
                           ]
dfUfoSubCmd7 = get_ghl2_table('ccb_ci_ufo_scmd7', 
                              inputParamGhl2UfoSubCmd7, 
                              'hddsn, enddate, testcode, mfgid, hddtrial, qualifier, lhd, test_band, tpmtpcdbl, tpmtpcod, tpmtpcid, otrc, finkfci, finktpi, defaultrate, procid, enddt, \
                               row_number() over (partition by hddsn, testcode, qualifier, lhd, test_band order by enddate desc) as row_num')
#dfUfoSubCmd7 = pd.read_pickle('df_ccb_ci_ufo_scmd7.pkl')

dfUfoSubCmd7.query("qualifier in ['3000', '9000', '4000', 'K000']", inplace=True)
dfUfoSubCmd7.loc[:,'tp_nm']  = (dfUfoSubCmd7.loc[:,'defaultrate'].values / 4096.0 * 25.4/stpi * 1e3).astype(np.float64)
if dfUfoSubCmd7.query("qualifier in ['3000', '9000']").shape[0] > 0:
    dfUfoSubCmd7.loc[(dfUfoSubCmd7['qualifier']=='3000') | (dfUfoSubCmd7['qualifier']=='9000'),'dtpi'] = dtpi_cmr
if dfUfoSubCmd7.query("qualifier in ['4000', 'K000']").shape[0] > 0:    
    dfUfoSubCmd7.loc[(dfUfoSubCmd7['qualifier']=='4000') | (dfUfoSubCmd7['qualifier']=='K000'),'dtpi'] = dtpi_smr

dfUfoSubCmd7.loc[:,'otrc'] = (dfUfoSubCmd7.loc[:,'otrc'].values * 25.4/dfUfoSubCmd7.loc[:,'dtpi'].values).astype(np.float64)     
dfUfoSubCmd7.loc[:,'tpf_dbl'] = (dfUfoSubCmd7.loc[:,'tpmtpcdbl'].values / 10000.0 * 25.4/dfUfoSubCmd7.loc[:,'dtpi'].values * 1e3).astype(np.float64)
dfUfoSubCmd7.loc[:,'tpf_od'] = (dfUfoSubCmd7.loc[:,'tpmtpcod'].values / 10000.0 * 25.4/dfUfoSubCmd7.loc[:,'dtpi'].values * 1e3).astype(np.float64)
dfUfoSubCmd7.loc[:,'tpf_id'] = (dfUfoSubCmd7.loc[:,'tpmtpcid'].values / 10000.0 * 25.4/dfUfoSubCmd7.loc[:,'dtpi'].values * 1e3).astype(np.float64)

#valid calc for only SMR
dfUfoSubCmd7.loc[:,'tpf_smr'] = (dfUfoSubCmd7.loc[:,'tpf_od'].values + dfUfoSubCmd7.loc[:,'tpf_id']).astype(np.float64)
dfUfoSubCmd7.loc[:,'sqm_smr'] = (dfUfoSubCmd7.loc[:,'tp_nm'].values - dfUfoSubCmd7.loc[:,'tpf_smr'].values).astype(np.float64)                            

#valid calc for only CMR
dfUfoSubCmd7.loc[:,'sqm_dbl'] = (dfUfoSubCmd7.loc[:,'tp_nm'].values - dfUfoSubCmd7.loc[:,'tpf_dbl'].values).astype(np.float64)
dfUfoSubCmd7.loc[:,'sqm_od']  = (dfUfoSubCmd7.loc[:,'tp_nm'].values - dfUfoSubCmd7.loc[:,'tpf_od'].values).astype(np.float64)
dfUfoSubCmd7.loc[:,'sqm_id']  = (dfUfoSubCmd7.loc[:,'tp_nm'].values - dfUfoSubCmd7.loc[:,'tpf_id'].values).astype(np.float64)

dfUfoSubCmd7.to_pickle('dfUfoSubCmd7.pkl')

"""
dfSqmFlat = pd.pivot_table(dfUfoSubCmd7.query("row_num==1"), 
                               index=['hddsn', 'mfgid', 'hddtrial', 'testcode', 'lhd'], 
                               columns=['qualifier', 'test_band'], 
                               values=['tp_nm', 'tpf_dbl', 'tpf_od', 'tpf_id', 'tpf_smr', 'sqm_dbl', 'sqm_od', 'sqm_id', 'sqm_smr', 'otrc', 'finkfci', 'finktpi'], 
                               aggfunc=np.sum).reset_index(drop=False)
        
cols = [x[0] for x in list(dfSqmFlat.columns)[:5]]+[str(x[0])+'_'+str(x[1])+'_'+str(x[2]) for x in list(dfSqmFlat.columns)[5:]]
dfSqmFlat.columns = cols
del dfUfoSubCmd7, cols
print('dfSqmFlat')
"""
dfUfoSubCmd7

current sql is select hddsn, enddate, testcode, mfgid, hddtrial, qualifier, lhd, test_band, tpmtpcdbl, tpmtpcod, tpmtpcid, otrc, finkfci, finktpi, defaultrate, procid, enddt,                                row_number() over (partition by hddsn, testcode, qualifier, lhd, test_band order by enddate desc) as row_num from ghl2.ccb_ci_ufo_scmd7 where product='pdq' and testcode in ('PDQC3EKM', 'PDQX073M') and enddt between '20201204' and '20210204' and hddtrial in ('M1TJ', 'M1U2', 'M1U3', '0000') and hddsn in ('2FA0A3AD', '2FA0A43D', '2FA0A4BD', '2FA0A59N', '2FA0A5NN', '2FA0A5VN', '2FA0A2WD', '2FA0A2LD', '2FA0A62N', '2FA09ZSF', '2FA0A63N', '2FA0A6HN', '2FA0A3ZD', '2FA0A37D', '2FA0A2XD', '2FA0A1ZD', '2FA0A4SN', '2FA0A4DD', '2FA0A4HD', '2FA0A3RD', '2FA0A6SN', '2FA0A20D', '2FA0A2RD', '2FA0A17F', '2FA0A5YN', '2FA0A6RN', '2FA0A36D', '2FA0A6GN', '2FA0A3HD', '2FA0A4PN', '2FA0A32D', '2FA0A4MN', '2FA0A6EN', '2FA0A3ND', '2FA0A0VF', '2FA0A3LD', '2FA0A23D', '2FA0A2TD', '2FA0A40D', '2FA0A6XN', '2FA0A3TD'

,hddsn,enddate,testcode,mfgid,hddtrial,qualifier,lhd,test_band,tpmtpcdbl,tpmtpcod,...,tp_nm,dtpi,tpf_dbl,tpf_od,tpf_id,tpf_smr,sqm_smr,sqm_dbl,sqm_od,sqm_id
0,2FA09ZHF,20201205030003,PDQX073M,KKBF04,M1TJ,3000,8,60,8655,6794,...,52.089844,500.0,43.96740,34.51352,36.52520,71.03872,-18.948876,8.122444,17.576324,15.564644
1,2FA09ZHF,20201205030003,PDQX073M,KKBF04,M1TJ,3000,9,33,9303,7342,...,53.886045,500.0,47.25924,37.29736,36.82492,74.12228,-20.236235,6.626805,16.588685,17.061125
2,2FA09ZHF,20201214160858,PDQX073M,KKBF04,M1TJ,9000,0,13,8562,7257,...,50.336409,500.0,43.49496,36.86556,33.10636,69.97192,-19.635511,6.841449,13.470849,17.230049
3,2FA09ZSF,20210126112408,PDQC3EKM,KKBF04,0000,3000,1,62,8806,7097,...,52.816878,500.0,44.73448,36.05276,38.23716,74.28992,-21.473042,8.082398,16.764118,14.579718
4,2FA09ZSF,20201205021841,PDQX073M,KKBF04,M1TJ,3000,0,12,9059,7291,...,52.859650,500.0,46.01972,37.03828,32.20720,69.24548,-16.385830,6.839930,15.821370,20.652450
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
230331,2FA0A6XN,20210203232034,PDQC3EKM,KQBF04,0000,9000,7,52,8461,7165,...,49.523842,500.0,42.98188,36.39820,36.86556,73.26376,-23.739918,6.541962,13.125642,12.658282
230332,2FA0A6XN,20210203232034,PDQC3EKM,KQBF04,0000,9000,17,47,8731,7038,...,50.550242,500.0,44.35348,35.75304,36.05276,71.80580,-21.255558,6.196762,14.797202,14.497482
230333,2FA0A6XN,20201205001910,PDQX073M,KQBF04,M1U3,3000,7,22,8672,7502,...,50.421942,500.0,44.05376,38.11016,34.64560,72.75576,-22.333818,6.368182,12.311782,15.776342
230334,2FA0A6XN,20201205001910,PDQX073M,KQBF04,M1U3,3000,8,25,9025,6862,...,52.774111,500.0,45.84700,34.85896,33.91408,68.77304,-15.998929,6.927111,17.915151,18.860031


In [31]:
# UFO ADC, BPI, TPI
inputParamGhl2UfoHdBandArray = [snlist_csv, 
                                listTestcode,
                                listEnddt, 
                                listHddTrial,
                                ['6400'],
                                ['1000', '2000']
                               ]
dfUfoHdBandArray = get_ghl2_table('ccb_ci_ufo_hd_band_array', inputParamGhl2UfoHdBandArray, 
                                  'hddsn, enddt, pfcode, testcode, mfgid, hddtrial, qualifier, lhd, band, adc, adckbpi, adcktpi, \
                                   row_number() over (partition by hddsn, testcode, qualifier, lhd, band order by enddate desc) as row_num')
dfUfoHdBandArray = pd.read_pickle('df_ccb_ci_ufo_hd_band_array.pkl')
dfUfoHdBandArray.loc[:,'adc']     = dfUfoHdBandArray.loc[:,'adc'].values.astype(np.float64)
dfUfoHdBandArray.loc[:,'adckbpi'] = dfUfoHdBandArray.loc[:,'adckbpi'].values.astype(np.float64)
dfUfoHdBandArray.loc[:,'adcktpi'] = dfUfoHdBandArray.loc[:,'adcktpi'].values.astype(np.float64)
dfUfoHdBandArray.to_pickle('dfUfoHdBandArray.pkl')

"""
dfAdcFlat = pd.pivot_table(dfUfoHdBandArray.query("band<7 and row_num==1"), 
                          index=['hddsn', 'mfgid', 'hddtrial', 'testcode', 'lhd'], 
                          columns=['qualifier', 'band'], 
                          values=['adc', 'adckbpi', 'adcktpi'], 
                          aggfunc=np.sum).reset_index(drop=False)
cols = [x[0] for x in list(dfAdcFlat.columns)[:5]]+[str(x[0])+'_'+str(x[1])+'_'+str(x[2]) for x in list(dfAdcFlat.columns)[5:]]
dfAdcFlat.columns = cols
del dfUfoHdBandArray, cols
print('dfAdcFlat')
"""
dfUfoHdBandArray

current sql is select hddsn, enddt, pfcode, testcode, mfgid, hddtrial, qualifier, lhd, band, adc, adckbpi, adcktpi,                                    row_number() over (partition by hddsn, testcode, qualifier, lhd, band order by enddate desc) as row_num from ghl2.ccb_ci_ufo_hd_band_array where product='pdq' and testcode in ('PDQC3EKM', 'PDQX073M') and enddt between '20201204' and '20210204' and hddtrial in ('M1TJ', 'M1U2', 'M1U3', '0000') and hddsn in ('2FA0A3AD', '2FA0A43D', '2FA0A4BD', '2FA0A59N', '2FA0A5NN', '2FA0A5VN', '2FA0A2WD', '2FA0A2LD', '2FA0A62N', '2FA09ZSF', '2FA0A63N', '2FA0A6HN', '2FA0A3ZD', '2FA0A37D', '2FA0A2XD', '2FA0A1ZD', '2FA0A4SN', '2FA0A4DD', '2FA0A4HD', '2FA0A3RD', '2FA0A6SN', '2FA0A20D', '2FA0A2RD', '2FA0A17F', '2FA0A5YN', '2FA0A6RN', '2FA0A36D', '2FA0A6GN', '2FA0A3HD', '2FA0A4PN', '2FA0A32D', '2FA0A4MN', '2FA0A6EN', '2FA0A3ND', '2FA0A0VF', '2FA0A3LD', '2FA0A23D', '2FA0A2TD', '2FA0A40D', '2FA0A6XN', '2FA0A3TD', '2FA0A2KD', '2FA0A27D', '2FA0A4ZN', '2FA0A5BN', '2

,hddsn,enddt,pfcode,testcode,mfgid,hddtrial,qualifier,lhd,band,adc,adckbpi,adcktpi,row_num
0,2FA0A20D,20210126,0000,PDQC3EKM,KQBF04,0000,1000,16,5,1107.0,2223.0,498.0,1
1,2FA0A2LD,20201205,0000,PDQX073M,KQBF04,M1U2,1000,2,1,931.0,2091.0,445.0,1
2,2FA0A2TD,20210126,0000,PDQC3EKM,KQBF04,0000,1000,1,4,1071.0,2409.0,445.0,1
3,2FA0A36D,20201204,0000,PDQX073M,KQBF04,M1U2,1000,14,6,1107.0,2428.0,456.0,1
4,2FA0A3ND,20210126,0000,PDQC3EKM,KQBF04,0000,1000,6,6,1093.0,2421.0,451.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
14539,2FA0A5XN,20201205,0000,PDQX073M,KQBF04,M1U3,1000,0,0,838.0,1744.0,481.0,1
14540,2FA0A5YN,20210126,0000,PDQC3EKM,KQBF04,0000,1000,2,6,1094.0,2385.0,459.0,1
14541,2FA0A62N,20210126,0000,PDQC3EKM,KQBF04,0000,1000,9,3,1070.0,2326.0,460.0,1
14542,2FA0A6SN,20201205,0000,PDQX073M,KQBF04,M1U3,1000,12,6,1140.0,2339.0,487.0,1


# RWDC

In [37]:
#rwdc
inputParamGhl2Dati = [snlist_csv, 
                     listTestcode,
                     listEnddt, 
                     listHddTrial,
                     ['6600'],
                     ['60H0', 'G0H0']
                    ]
dfRwdc = get_ghl2_table('ccb_ci_rwdc', inputParamGhl2Dati, 
                        'hddsn, enddate, lhd, qualifier, phd, testcode, mfgid, hddtrial, band, rwdcmin, rwdcmax, \
                         row_number() over (partition by hddsn, qualifier, lhd, band order by enddate desc) as row_num')
dfRwdc = pd.read_pickle('df_ccb_ci_rwdc.pkl')
#dfRwdc.loc[:,['hddsn', 'lhd', 'phd', 'testcode', 'mfgid', 'band', 'rwdcmin', 'rwdcmax']]
dfRwdcFlat = pd.pivot_table(dfRwdc.query("row_num==1"),
                            index=['hddsn', 'mfgid', 'hddtrial', 'testcode', 'lhd'], 
                            columns=['qualifier', 'band'], 
                            values=['rwdcmin', 'rwdcmax'], 
                            aggfunc=np.sum).reset_index(drop=False)
cols = [x[0] for x in list(dfRwdcFlat.columns)[:5]]+[str(x[0])+'_'+str(x[1])+'_'+str(x[2]) for x in list(dfRwdcFlat.columns)[5:]]
dfRwdcFlat.columns = cols
dfRwdcFlat.to_pickle('dfRwdcFlat.pkl')

current sql is select hddsn, enddate, lhd, qualifier, phd, testcode, mfgid, hddtrial, band, rwdcmin, rwdcmax,                          row_number() over (partition by hddsn, qualifier, lhd, band order by enddate desc) as row_num from ghl2.ccb_ci_rwdc where product='pdq' and testcode in ('PDQX083M') and enddt between '20201228' and '20210107' and hddsn in ('2FA00EPA', '2FA01HYA', '2FA026TA', '2FA00E8A', '2FA01K0A', '2FA026YA', '2FA00EWA', '2FA00ESA', '2FA00EYA', '2FA0093A') and procid in ('6600') and qualifier in ('60H0', 'G0H0') and pfcode not in ('9999', '99V6', '99V7') 
(8192, 12)


In [38]:
# Merge all tables

dfParamAll = dfHeadMap.copy(deep = True)
for df in [dfRsbnFlat, dfSerFlat, dfSerOffsetFlat, 
           dfOwFlat, dfMcwFlat, dfMrwFlat, dfAmpFlat, 
           dfAsymFlat, dfRwipFlat, dfSatiSerFlat, dfNmaxTempFlat, dfAsb]:
    dfParamAll = dfParamAll.merge(df,
                             left_on  = ['hddsn', 'lhd', 'phd', 'mfgid', 'testcode'],
                             right_on = ['hddsn', 'lhd', 'phd', 'mfgid', 'testcode'],
                             how='left',
                            )
    print(dfParamAll.shape)
    
for df in [dfSqmFlat, dfAdcFlat]:
    dfParamAll = dfParamAll.merge(df,
                             left_on  = ['hddsn', 'lhd', 'mfgid', 'testcode'],
                             right_on = ['hddsn', 'lhd', 'mfgid', 'testcode'],
                             how='left',
                            )
    print(dfParamAll.shape)

dfParamAll.to_pickle('dfParamAll.pkl')
dfParamAll.to_csv('dfParamAll.csv')

NameError: name 'dfSerFlat' is not defined

In [ ]:
pd.pivot_table(dfParamAll, index=['testcode', 'mfgid'], columns='lhd', values='hddsn', aggfunc='count')

# DET Data

In [15]:
# Association

inputParamVqaa = [snlist_csv, 
                  listTestcode,
                  listEnddt, 
                  listHddTrial,
                  ['6400']
                 ]
get_vqaa_table ('fact_hdd_headops', inputParamVqaa, 'product, slidersn, pheadno, hddsn, procid, mfgid, hddtrial, enddate, \
                 row_number() over ( partition by slidersn order by enddate desc) as row_num', whereAdded = None)

current sql is select product, slidersn, pheadno, hddsn, procid, mfgid, hddtrial, enddate,                  row_number() over ( partition by slidersn order by enddate desc) as row_num from vqaa.fact_hdd_headops where product='pdq' and testpgmver in ('PDQX083M') and enddt between '20201223' and '20210101' and hddtrial in ('Z07E', 'NPIF') and hddsn in ('2FA00WRA', '2FA02E8A', '2FA06EXA', '2FA06GNA', '2FA06H4A', '2FA06H1A', '2FA06G5A', '2FA06G3A', '2FA02DKA', '2FA01E4A', '2FA02EUA', '2FA02EAA', '2FA02DNA', '2FA01EHA') and procid in ('6400') 
(306, 9)


,product,slidersn,pheadno,hddsn,procid,mfgid,hddtrial,enddate,row_num
0,pdq,49FB51932F,7,2FA02DKA,6400,KJ1C03,Z07E,20201223104205,1
1,pdq,4AF083B607,4,2FA06G3A,6400,KJ1C03,NPIF,20201229143527,1
2,pdq,49FBF2D223,12,2FA02DKA,6400,KJ1C03,Z07E,20201223104205,1
3,pdq,49FBF1B432,0,2FA02DKA,6400,KJ1C03,Z07E,20201223104205,1
4,pdq,49F8D1182C,10,2FA06H4A,6400,KJ1C03,NPIF,20201229142426,1
...,...,...,...,...,...,...,...,...,...
301,pdq,49F491C709,7,2FA01EHA,6400,KJ1C03,Z07E,20201223225255,1
302,pdq,49F6A28B38,7,2FA02DNA,6400,KJ1C03,Z07E,20201223224858,1
303,pdq,4AFF60B71B,7,2FA06GNA,6400,KJ1C03,NPIF,20201229141542,1
304,pdq,4AFF60B71B,7,2FA06GNA,6400,KJ1C03,NPIF,20201228132753,2


In [19]:
## sdet ##
inputParamSldr = ['df_fact_hdd_headops.csv', 
                  ['2020-01-01', '2020-07-31'],
                 ]
get_sdet_table('sldr.sdet_special_trans', inputParamSldr, 
               'slidersn, product, storeday, \
                wew_r0_nm_sto_2, wew_r1_nm_sto_2, wew_r0_mww_nm_sto_2, wew_r1_mww_nm_sto_2, wew_r0_umrw_nm_sto_2, wew_r1_umrw_nm_sto_2, wew_r0_eb_nm_sto_2, wew_r1_eb_nm_sto_2, \
                row_number() over ( partition by slidersn order by storeday desc) as row_num',
                whereAdded = None)

current sql is select slidersn, product, storeday,                 wew_r0_nm_sto_2, wew_r1_nm_sto_2, wew_r0_mww_nm_sto_2, wew_r1_mww_nm_sto_2, wew_r0_umrw_nm_sto_2, wew_r1_umrw_nm_sto_2, wew_r0_eb_nm_sto_2, wew_r1_eb_nm_sto_2,                 row_number() over ( partition by slidersn order by storeday desc) as row_num from hive.sldr.sdet_special_trans where storeday between '2020-01-01' and '2020-07-31' and slidersn in ('49FB51932F', '4AF083B607', '49FBF2D223', '49FBF1B432', '49F8D1182C', '4AF081CC0A', '4AF081CC0A', '4AF081CC0A', '49F8D11835', '49FBF3FF22', '4AF0823428', '4AFF60B339', '49FBF3A615', '4AFF61130A', '4AFF61130A', '49F6A28627', '49F8D1183B', '49FBF0330B', '49F8818902', '4AF080AC1D', '4AF080AC1D', '4AF080AC1D', '4AFF60B338', '4AF0821A19', '4AFF605126', '49F491C70C', '49FBF2D221', '49FBF0F215', '49F491C715', '49FBF03510', '49FCC2130D', '49FCC2130D', '4AFF61591C', '4AFF61591C', '49F5508532', '49F5536420', '49F8A33C1D', '4A2B82810B', '49F4910703', '49FBF03B0F', '49FBF0351D', '4

,slidersn,product,storeday,wew_r0_nm_sto_2,wew_r1_nm_sto_2,wew_r0_mww_nm_sto_2,wew_r1_mww_nm_sto_2,wew_r0_umrw_nm_sto_2,wew_r1_umrw_nm_sto_2,wew_r0_eb_nm_sto_2,wew_r1_eb_nm_sto_2,row_num
0,4AF0823428,C3_B_D,2020-07-29,None,None,None,None,None,None,None,None,1
1,4AFF60B339,C3_T_D,2020-07-28,None,None,None,None,None,None,None,None,1
2,49F8A33C1D,C3_B_D,2020-03-10,None,None,None,None,None,None,None,None,1
3,4AFF61591C,C3_T_D,2020-07-30,None,None,None,None,None,None,None,None,1
4,49F8D11835,C3_B_D,2020-06-07,None,None,None,None,None,None,None,None,1
...,...,...,...,...,...,...,...,...,...,...,...,...
86,49FBF2DD0F,C3_T_D,2020-03-10,None,None,None,None,None,None,None,None,1
0,49F491C709,C3_T_D,2020-03-11,None,None,None,None,None,None,None,None,1
1,49F6A28B38,C3_T_D,2020-03-12,None,None,None,None,None,None,None,None,1
2,4AFF60B71B,C3_T_D,2020-07-28,None,None,None,None,None,None,None,None,1


In [20]:
#Combine slider sn, pheadno, and HDD SN
df = pd.read_pickle('df_fact_hdd_headops.pkl').query("row_num==1").merge(pd.read_pickle('df_sldr.sdet_special_trans.pkl').loc[:,['slidersn', 'storeday', 'wew_r0_nm_sto_2', 'row_num']].query("row_num==1"),
                                                                         left_on=['slidersn', 'row_num'], 
                                                                         right_on=['slidersn', 'row_num'], 
                                                                         how='left')
df.to_pickle('hddops_association.pkl')
df

,product,slidersn,pheadno,hddsn,procid,mfgid,hddtrial,enddate,row_num,storeday,wew_r0_nm_sto_2
0,pdq,49FB51932F,7,2FA02DKA,6400,KJ1C03,Z07E,20201223104205,1,2020-03-09,None
1,pdq,4AF083B607,4,2FA06G3A,6400,KJ1C03,NPIF,20201229143527,1,2020-07-28,None
2,pdq,49FBF2D223,12,2FA02DKA,6400,KJ1C03,Z07E,20201223104205,1,2020-03-14,None
3,pdq,49FBF1B432,0,2FA02DKA,6400,KJ1C03,Z07E,20201223104205,1,2020-03-14,None
4,pdq,49F8D1182C,10,2FA06H4A,6400,KJ1C03,NPIF,20201229142426,1,2020-06-07,None
...,...,...,...,...,...,...,...,...,...,...,...
247,pdq,49FB530725,15,2FA02DKA,6400,KJ1C03,Z07E,20201223104205,1,2020-03-09,None
248,pdq,49F8A34E11,10,2FA02E8A,6400,KJ1C03,Z07E,20201223223939,1,2020-03-08,None
249,pdq,49F491C709,7,2FA01EHA,6400,KJ1C03,Z07E,20201223225255,1,2020-03-11,None
250,pdq,49F6A28B38,7,2FA02DNA,6400,KJ1C03,Z07E,20201223224858,1,2020-03-12,None
